In [1]:
!pip install -I pillow==5.1.0

    100% |████████████████████████████████| 2.0MB 1.9MB/s 


In [1]:
# this sets up the virtual environment if gdrive doesn't contain the repo clone
# it by setting the the if to true and modifie the path as you wish
#@title Default title text
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
import os
!pip install -q xlrd
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/ADL4CV')
#!ls
if False:
  from getpass import getpass
  user = getpass('GitHub user')
  password = getpass('GitHub password')
  os.environ['GitHub_AUTH'] = user + ':' + password
  !git clone https://$GitHub_AUTH@github.com/Cram13/ADL4CV_Project.git
os.chdir('ADL4CV_Project')
!git pull origin master
#!pip install -I pillow==5.1.0
!pip3 install torch torchvision
!pip3 install tensorboardX
#!pip install PIL
#!pip install image
#import PIL.image

--2019-02-06 18:43:59--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.201.75.180, 52.86.186.182, 52.3.53.115, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.201.75.180|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  19.2MB/s    in 0.3s    

2019-02-06 18:44:00 (19.2 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.

In [2]:
 #!git checkout origin/master
!git pull origin master


#!git status
#!git fetch
#!git reset --hard origin/master

From https://github.com/Cram13/ADL4CV_Project
 * branch            master     -> FETCH_HEAD
Already up to date.


In [0]:
import torch.nn as nn
from yolo.yolo_encoder import YoloEncoder
from Flow import FlowNetSEncoder
from lstm import ConvLSTMCell
import torchvision.transforms as transforms
from yolo.yolo_utils import *
import torch.nn.functional as F

class Opt(object):
    def __init__(self):
        self.batch_size = 1
        self.reduction = 32
        self.num_epoches = 20
        self.momentum = 0.9
        self.decay = 0.0005
        self.image_size = 416
        self.encoding_size = self.image_size // self.reduction
        self.log_path = './log/flow'
        self.pre_trained_yolo_path = './models/yolo80_coco.pt'
        self.pre_trained_flownet_path = './models/flownets_EPE1.951.pth.tar'
        self.dataset_file = 'dataset_utils/Mot17_test_single.txt'
        self.learning_rate = 3e-5
        self.useCuda = True  # for easy disabeling cuda
        self.num_workers = 4
        self.reinitailise_last_layer = True
        self.model = None
        self.decoder = None
        self.yolo_loss = None
        self.pred_loss = None
        self.optimizer_encoder = None
        self.optimizer_decoder = None
        self.conf_threshold = 0.25

    def device(self):
        return torch.device('cuda') if self.useCuda else torch.device('cpu')

class YoloLSTM_part(nn.Module):
    def __init__(self, input_size=(13, 13), input_dim=1024, hidden_dim=1024, batch_size=1,
                 num_anchors=5):
        super(YoloLSTM_part, self).__init__()

        self.lstm_cell = ConvLSTMCell(input_size=input_size, input_dim=input_dim, hidden_dim=hidden_dim,
                                                            kernel_size=(3, 3), bias=True)
        self.reinit_lstm(batch_size)
        self.stage3_conv2 = nn.Conv2d(1024, num_anchors * 5, 1, 1, 0, bias=False)

    def forward(self, input):
        self.hidden, self.cell = self.lstm_cell(input, (self.hidden, self.cell))
        output = self.hidden
        output = self.stage3_conv2(output)
        return output

    def reinit_lstm(self, batch_size):
        dev = next(self.parameters()).device
        self.hidden, self.cell = (Variable(torch.zeros(batch_size, self.lstm_cell.hidden_dim,
                                                        self.lstm_cell.height, self.lstm_cell.width).to(dev), requires_grad=True),
                                  Variable(torch.zeros(batch_size, self.lstm_cell.hidden_dim,
                                                        self.lstm_cell.height, self.lstm_cell.width).to(dev), requires_grad=True))

class YoloFlowLSTM(nn.Module):
    def __init__(self, batch_size, image_size=416, freeze_encoder=True,
                 anchors=[(0.215, 0.8575), (0.3728125, 1.8225), (0.621875, 2.96625),
                          (1.25, 6.12), (3.06125, 11.206875)]):
        super(YoloFlowLSTM, self).__init__()
        self.image_size = image_size
        self.anchors = anchors
        self.encoder = YoloEncoder()
        self.flownet = FlowNetSEncoder(False)

        # reduce flow dimensions to 13 by 13 with 512 channels
        self.conv_flow = nn.Conv2d(512 + 2, 512, kernel_size=2, bias=True)  # not sure about bias
        if freeze_encoder:
            for param in self.encoder.parameters():
                param.requires_grad = False
            for param in self.flownet.parameters():
                param.requires_grad = False
        self.lstm_part = YoloLSTM_part(input_dim=1024+512, batch_size=batch_size)  # todo set the right dimension

    def forward(self, input):
        flow_input, yolo_input = input
        yolo_encoding = self.encoder.forward(yolo_input)
        flow_encoding = self.flownet(flow_input)
        flow_encoding = self.conv_flow(flow_encoding)

        encoding = torch.cat((yolo_encoding, flow_encoding), 1) # todo
        output = self.lstm_part.forward(encoding)
        return output

    def reinit_lstm(self, batch_size):
        self.lstm_part.reinit_lstm(batch_size)
        
class BigYoloFlowLSTM(nn.Module):
    def __init__(self, batch_size, image_size=832, freeze_encoder=True,
                 anchors = [(0.43, 1.715), (0.745625, 3.645), 
                            (1.24375, 5.9325), (2.5, 12.24), 
                            (6.1225, 22.41375)]):
        super(BigYoloFlowLSTM, self).__init__()
        self.image_size = image_size
        self.anchors = anchors
        self.encoder = YoloEncoder()
        self.flownet = FlowNetSEncoder(False, image_size=832)

        self.conv_flow = nn.Conv2d(512 + 2 + 512, 512, kernel_size=1, bias=True)
        if freeze_encoder:
            for param in self.encoder.parameters():
                param.requires_grad = False
            for param in self.flownet.parameters():
                param.requires_grad = False
        self.lstm_part = YoloLSTM_part(input_size=(26,26), input_dim=1024+512, batch_size=batch_size)  # todo set the right dimension

    def forward(self, input):
        flow_input, yolo_input = input
        yolo_encoding = self.encoder.forward(yolo_input)
        flow_encoding = self.flownet(flow_input)
        flow_encoding = self.conv_flow(flow_encoding)

        encoding = torch.cat((yolo_encoding, flow_encoding), 1) # todo
        output = self.lstm_part.forward(encoding)
        return output

    def reinit_lstm(self, batch_size):
        self.lstm_part.reinit_lstm(batch_size)

class ConvLSTM(nn.Module):
    def __init__(self, input_size, input_dim, hidden_dim, batch_size_init):
        super(ConvLSTM, self).__init__()
        # layer parameter

        self.input = nn.Conv2d(input_dim, 512, kernel_size=3, padding=1)

        self.lstm = ConvLSTMCell(input_size, 512, hidden_dim, kernel_size=(3, 3), bias=True)

        self.output = nn.Conv2d(hidden_dim, input_dim, kernel_size=3, padding=1)

        self.hidden, self.cell = self.lstm.init_hidden(batch_size_init)
        

    def forward(self, input):

        #x = F.relu(self.input(input))
        x = self.input(input)# todo bigger embedding

        self.hidden, self.cell = self.lstm(x.squeeze(dim=2), (self.hidden, self.cell))

        return self.output(self.hidden)

    def reset_hidden(self, batch_size):
        self.hidden, self.cell = self.lstm.init_hidden(batch_size)

    def set_hidden(self, hidden, cell):
        self.hidden = hidden
        self.cell = cell







In [0]:
from dataset_utils.datasets.MotBBImageSequence import *
from torch.utils.data import DataLoader, Subset
from yolo.loss import YoloLoss
from yolo.yolo_utils import logits_to_box_params
import torch
from Flow import preprocess
from tensorboardX import SummaryWriter
import shutil
from train_yolo import writeLossToSummary
from lstm.LSTMLosses import NaiveLoss, displacement_error, prediction_to_box_list, center_distance
import cv2


def filter_gt(gt):
    # used to filter only existing bb for loss
    t = gt[gt[:, 0] != 0.0]
    return t[:, 1:]


def draw_pred_sequence(box_list, images, seq_length, obs_length, name, image_size=416):

    # draw the predicted path
    image = images[obs_length]
    image = misc.imresize(image, (image_size, image_size))
    for i in range(seq_length):
        boxes_in, boxes_pred, boxes_tar = box_list[i]
        boxes_pred = np.squeeze(boxes_pred) * image_size
        boxes_tar = np.squeeze(boxes_tar) * image_size

        # draw predicted boxes
        for box in range(len(boxes_pred)):
            if np.sum(boxes_pred[box, 1:]) != 0:

                x = max(min(boxes_pred[box, 1], image_size-1), 0)
                y = max(min(boxes_pred[box, 2], image_size-1), 0)
                if box == 0:
                    image[int(y), int(x)] = [255, 255, 0]
                else:
                    image[int(y), int(x)] = [255, 0, 0]

        # draw target boxes
        for box in range(len(boxes_tar)):
            if np.sum(boxes_tar[box, 0]) != 0:

                x = max(min(boxes_tar[box, 1], image_size-1), 0)
                y = max(min(boxes_tar[box, 2], image_size-1), 0)

                image[int(y), int(x)] = [0, 255, 0]
    
    misc.imsave(name, image)
    
def draw_pred_sequence_2(box_list, images, seq_length, obs_length, name, image_size=416):
    # draw the predicted path
    # first goal is to get an actual path
    image = images[obs_length]
    image = misc.imresize(image, (image_size, image_size))
    for i in range(seq_length):
        boxes_in, boxes_pred, boxes_tar = box_list[i]
        boxes_pred = np.squeeze(boxes_pred) * image_size
        boxes_tar = np.squeeze(boxes_tar) * image_size

        # draw predicted boxes
        for box in range(len(boxes_pred)):
            if np.sum(boxes_pred[box, 1:]) != 0:

                x = max(min(boxes_pred[box, 1], image_size-1), 0)
                y = max(min(boxes_pred[box, 2], image_size-1), 0)
                if i != 0:
                    cv2.circle(image, (x, y), radius=2, color=(255, 0, 0, (1.0/seq_length) * (i+1)), thickness=-1)
                else:
                    cv2.circle(image, (x, y), radius=2, color=(255, 0, 255, (1.0/seq_length) * (i+1)), thickness=-1)

        # draw target boxes
        for box in range(len(boxes_tar)):
            if np.sum(boxes_tar[box, 0]) != 0:
                x = max(min(boxes_tar[box, 1], image_size-1), 0)
                y = max(min(boxes_tar[box, 2], image_size-1), 0)
                #image[int(y), int(x)] = [0, 255, 0]
                if i != 0:
                    cv2.circle(image, (x, y), radius=2, color=(0, 255, 0, (1.0/seq_length) * (i+1)), thickness=-1)
                else:
                    cv2.circle(image, (x, y), radius=2, color=(255, 255, 0, (1.0/seq_length) * (i+1)), thickness=-1)


    misc.imsave(name, image)
    
def load_Snapshot_to_yolo_LSTM(model, opt):
    def remove_flownet_weights(dict):
        del dict["deconv4.0.weight"]
        del dict["deconv3.0.weight"]
        del dict["deconv2.0.weight"]

        del dict["predict_flow5.weight"]
        del dict["predict_flow4.weight"]
        del dict["predict_flow3.weight"]
        del dict["predict_flow2.weight"]

        del dict["upsampled_flow5_to_4.weight"]
        del dict["upsampled_flow4_to_3.weight"]
        del dict["upsampled_flow3_to_2.weight"]
        return dict



    model_state_dict_yolo = torch.load(opt.pre_trained_yolo_path, map_location=opt.device())['model_state_dict']
    model_state_dict_flow = torch.load(opt.pre_trained_flownet_path, map_location=opt.device())['state_dict']

    del model_state_dict_yolo["stage3_conv2.weight"]
    model_state_dict_flow = remove_flownet_weights(model_state_dict_flow)
    model.encoder.load_state_dict(model_state_dict_yolo, strict=True)
    model.flownet.load_state_dict(model_state_dict_flow, strict=True)
    
def mean_squared_trajectory_error(pred_box_list, metric, image_size=416):
    # shape (batch_size, 120, boxes) boxes: (5) id, x, y, w, h

    error = []
    for frame in pred_box_list:
        _, output, target = frame
        for batch_id in range(len(target)):
            for box, box_id in enumerate(target[batch_id, :, 0]):
                if box_id != 0:
                    target_box = np.where(output[batch_id, :, 0] == box_id)
                    target_box = output[batch_id, target_box[0][0], 1:] * image_size
                    error.append(metric(target_box, target[batch_id, box, 1:]*image_size))
    return np.mean(error)


fine tune yolo flow lstm without sequence prediction

In [0]:
opt = Opt()
opt.num_epoches = 100
opt.useCuda = True
opt.learning_rate = 3e-5
opt.decay = 0.0
opt.batch_size = 1
opt.model = YoloFlowLSTM(opt.batch_size)
opt.decoder = ConvLSTM((opt.encoding_size, opt.encoding_size), 5 * 4, 1024, opt.batch_size)
opt.pre_trained_yolo_path = 'models/snapshot0020.tar'
load_Snapshot_to_yolo_LSTM(opt.model, opt)
# all parameters are on the device the hidden states are not parameters
opt.model.to(opt.device())
opt.decoder.to(opt.device())
opt.yolo_loss = YoloLoss(opt.model.anchors, filter_fkt=filter_gt)
loss_params = {"grid_shape": (opt.encoding_size, opt.encoding_size),
               "image_shape": (opt.image_size, opt.image_size),
               "path_anchors": "dataset_utils/anchors/anchors5.txt"}
opt.pred_loss = NaiveLoss(loss_params)

In [0]:
def test_loss(pred, input, target):
    mask = target[:, :, :, :, 0] != 0
    #print(torch.sum(mask, dim=(0, 3)).detach().cpu().numpy())
    mask = torch.unsqueeze(mask.int(), 4)

    input = input[:, :, :, :, 1:]
    target = target[:, :, :, :, 1:]
    #print(f"dividend:  {torch.nonzero(mask).size(0)}")
    #print(torch.masked_select(pred, (pred.detach() * mask.detach() > 0).byte()))
    return torch.sum(mask.float() * torch.pow((target - input - pred), 2)) / torch.nonzero(mask).size(0)

def train(opt):
    # setup train and eval set
    torch.cuda.manual_seed(123)
    torch.manual_seed(123)
    
    # log stuff
    if os.path.isdir(opt.log_path):
        shutil.rmtree(opt.log_path)
    os.makedirs(opt.log_path)
    writer = SummaryWriter(opt.log_path)

    dataset = MotBBImageSequence('dataset_utils/Mot17_test_single.txt', use_only_first_video=False)
    train_data = Subset(dataset, range(0, dataset.valid_begin))
    valid_data = Subset(dataset, range(dataset.valid_begin, len(dataset)))
    train_loader = DataLoader(train_data, batch_size=opt.batch_size, shuffle=True, num_workers=opt.num_workers, drop_last=True)
    valid_loader = DataLoader(valid_data, batch_size=opt.batch_size, shuffle=False, num_workers=opt.num_workers, drop_last=True)

    opt.optimizer_encoder = torch.optim.Adam(opt.model.parameters(), lr=opt.learning_rate, betas=(opt.momentum, 0.999),
                                             weight_decay=opt.decay)
    opt.optimizer_decoder = torch.optim.Adam(opt.decoder.parameters(), lr=opt.learning_rate, betas=(opt.momentum, 0.999),
                                             weight_decay=opt.decay)

    obs_length = 10
    pred_length = 9
    
    len_pretraining = 0  # epochs
    epoch_len = len(train_loader)

    opt.model.train()
    opt.decoder.train()
    for img_num, (gt, b, img1, img2) in enumerate(train_loader):
        for epoch in range(opt.num_epoches):
            
            opt.model.lstm_part.reinit_lstm(opt.batch_size)
            opt.decoder.reset_hidden(opt.batch_size)
            opt.optimizer_encoder.zero_grad()
            opt.optimizer_decoder.zero_grad()
            
            seq_loss = 0
            seq_loss_coord, seq_loss_conf, seq_loss_pred = (0.0, 0.0, 0.0)
            pred_sequence = []
            for i in range(obs_length):  
                single_gt = b[:, i].to(opt.device())
                single_img2 = img2[:, i].to(opt.device())
                single_img2_normed = preprocess(img2[:, i]).to(opt.device())
                single_img1_normed = preprocess(img1[:, i]).to(opt.device())

                double_image = torch.cat((single_img1_normed, single_img2_normed), dim=1)

                logits = opt.model((double_image, single_img2))

                loss, loss_coord, loss_conf = opt.yolo_loss(logits, single_gt)
                seq_loss += loss
                seq_loss_conf += loss_conf.item()
                seq_loss_coord += loss_coord.item()
            if epoch >= len_pretraining:
                prev_out = logits_to_box_params(logits.detach(), opt.model.anchors)
                # the box parameters are normalized to [0, 1]
                # at the moment [batch, anchors, ...., h* w]
                # rearrange tensor s.t [batch, h, w, anchors, ...]
                # ... == x, y, w, h, conf
                #print(f"origin mask {torch.sum(mask, dim=(0, 1))}")
                prev_out = prev_out.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                                   .permute(0, 3, 4, 1, 2)
                # be carefull the position of the conf/id in the targets is 0 not 4
                # so we change it at this point to be consistent with the labels

                mask = (prev_out[:, :, :, :, 4] > 0.1).float()
                #print(f"origin mask {torch.sum(mask, dim=(0, 3))}")
                mask = mask.unsqueeze(-1)
                prev_out = torch.Tensor(np.roll(prev_out.cpu().numpy(), 1, axis=-1)).to(opt.device())

                opt.decoder.set_hidden(opt.model.lstm_part.hidden, opt.model.lstm_part.cell)
                for i in range(pred_length):
                    # todo anchor box assignment of the target boxes does not use iou
                    _, yolo_target = opt.pred_loss.to_yolo(input=gt[:, obs_length].numpy(), target=b[:, obs_length + i].numpy(), use_iou=True)
                    yolo_target = torch.Tensor(yolo_target).to(opt.device())
                    # target boxes are in [0, grid_h] -> normalize to 1
                    # at this point i assume that the image is a square !!!
                    yolo_target[:, :, :, :, 1:] = yolo_target[:, :, :, :, 1:] / opt.encoding_size
                    #if i == 0:
                      #print(f"target mask {torch.sum((yolo_target[:, :, :, :, 0] > 0.0).float(), dim=(0, 3))}")
                    prev_out = prev_out * mask  # todo maybe there is a better way
                    input = prev_out[:, :, :, :, 1:].contiguous()\
                                                    .view(opt.batch_size, len(opt.model.anchors)*4,
                                                          opt.encoding_size, opt.encoding_size)
                    pred = opt.decoder(input)
                    pred = pred.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                                   .permute(0, 3, 4, 1, 2)

                    seq_loss_pred += opt.pred_loss.forward(pred, prev_out, mask*yolo_target)
                    pred_sequence.append((prev_out.detach().cpu().numpy(),
                                          pred.detach().cpu().numpy(),
                                          (mask*yolo_target).detach().cpu().numpy()))
                    prev_out[:, :, :, :, 1:] += pred

                seq_loss += seq_loss_pred
            seq_loss.backward()
            opt.optimizer_encoder.step()
            
            seq_loss = seq_loss.item() if epoch <= len_pretraining else seq_loss.item() - seq_loss_pred.item()
            
            writeLossToSummary(writer, 'Train', seq_loss/obs_length, 
                                                seq_loss_coord / obs_length,
                                                seq_loss_conf / obs_length, 
                                                epoch * epoch_len + img_num)

            print(f"epoch:{epoch} it: {img_num}")
            print(f"loss_seq: {seq_loss/obs_length}, loss_coord: {seq_loss_coord / obs_length}, loss_conf: {seq_loss_conf / obs_length}")
            if epoch >= len_pretraining:
                seq_loss_pred = seq_loss_pred.item()
                box_list = prediction_to_box_list(pred_sequence)
                dis_error = displacement_error(box_list, center_distance)
                writer.add_scalar('Train/loss_pred', seq_loss_pred / pred_length, epoch * epoch_len + img_num)
                writer.add_scalar('Train/dis_err', dis_error / pred_length, epoch * epoch_len + img_num)
                print(f"loss_pred: {seq_loss_pred / pred_length}, dis_error: {dis_error / pred_length}")
                
                
        ###############
        # VALIDATION
        ###############
        print("###############")
        print("# VALIDATION BEGIN")
        print("###############")
        opt.model.eval()
        opt.decoder.eval()
        valid_len = len(valid_loader)
        loss_val = 0
        loss_coord_val = 0
        loss_conf_val = 0
        loss_pred_val = 0
        dis_error_val = 0
        for img_num, (gt, b, img1, img2) in enumerate(valid_loader):
            opt.model.lstm_part.reinit_lstm(opt.batch_size)
            opt.decoder.reset_hidden(opt.batch_size)
            opt.optimizer_encoder.zero_grad()
            opt.optimizer_decoder.zero_grad()
            
            seq_loss = 0
            seq_loss_coord, seq_loss_conf, seq_loss_pred = (0.0, 0.0, 0.0)
            pred_sequence = []
            for i in range(obs_length):
                single_gt = b[:, i].to(opt.device())
                single_img2 = img2[:, i].to(opt.device())
                single_img2_normed = preprocess(img2[:, i]).to(opt.device())
                single_img1_normed = preprocess(img1[:, i]).to(opt.device())

                double_image = torch.cat((single_img1_normed, single_img2_normed), dim=1)
                with torch.no_grad():
                    logits = opt.model((double_image, single_img2))

                    loss, loss_coord, loss_conf = opt.yolo_loss(logits, single_gt)
                seq_loss += loss
                seq_loss_conf += loss_conf.item()
                seq_loss_coord += loss_coord.item()
          
            if epoch >= len_pretraining:
                with torch.no_grad():
                    prev_out = logits_to_box_params(logits.detach(), opt.model.anchors)
                    # the box parameters are normalized to [0, 1]
                    # at the moment [batch, anchors, ...., h* w]
                    # rearrange tensor s.t [batch, h, w, anchors, ...]
                    # ... == x, y, w, h, conf
                    #print(f"origin mask {torch.sum(mask, dim=(0, 1))}")
                    prev_out = prev_out.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                                       .permute(0, 3, 4, 1, 2)
                    # be carefull the position of the conf/id in the targets is 0 not 4
                    # so we change it at this point to be consistent with the labels

                    mask = (prev_out[:, :, :, :, 4] > 0.1).float()
                    #print(f"origin mask {torch.sum(mask, dim=(0, 3))}")
                    mask = mask.unsqueeze(-1)
                    prev_out = torch.Tensor(np.roll(prev_out.cpu().numpy(), 1, axis=-1)).to(opt.device())

                    opt.decoder.set_hidden(opt.model.lstm_part.hidden, opt.model.lstm_part.cell)
                    for i in range(pred_length):
                        # todo anchor box assignment of the target boxes does not use iou
                        _, yolo_target = opt.pred_loss.to_yolo(input=gt[:, obs_length].numpy(), target=b[:, obs_length + i].numpy(), use_iou=True)
                        yolo_target = torch.Tensor(yolo_target).to(opt.device())
                        # target boxes are in [0, grid_h] -> normalize to 1
                        # at this point i assume that the image is a square !!!
                        yolo_target[:, :, :, :, 1:] = yolo_target[:, :, :, :, 1:] / opt.encoding_size
                        #if i == 0:
                          #print(f"target mask {torch.sum((yolo_target[:, :, :, :, 0] > 0.0).float(), dim=(0, 3))}")
                        prev_out = prev_out * mask  # todo maybe there is a better way
                        input = prev_out[:, :, :, :, 1:].contiguous()\
                                                        .view(opt.batch_size, len(opt.model.anchors)*4,
                                                              opt.encoding_size, opt.encoding_size)
                        pred = opt.decoder(input)
                        pred = pred.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                                       .permute(0, 3, 4, 1, 2)

                        seq_loss_pred += opt.pred_loss.forward(pred, prev_out, mask*yolo_target)
                        pred_sequence.append((prev_out.detach().cpu().numpy(),
                                              pred.detach().cpu().numpy(),
                                              (mask*yolo_target).detach().cpu().numpy()))
                        prev_out[:, :, :, :, 1:] += pred


                    seq_loss_pred = seq_loss_pred.item()
                    box_list = prediction_to_box_list(pred_sequence)
                    dis_error = displacement_error(box_list, center_distance)
            
            loss_val += seq_loss.item() / valid_len /obs_length
            loss_coord_val += seq_loss_coord / valid_len /obs_length
            loss_conf_val += seq_loss_conf / valid_len /obs_length
            if epoch >= len_pretraining:
                loss_pred_val += seq_loss_pred / valid_len  / pred_length
                dis_error_val += dis_error / valid_len  / pred_length
        
        writeLossToSummary(writer, 'Val', loss_val, loss_coord_val,
                           loss_conf_val, (epoch+1) * epoch_len)

        print(f"epoch:{epoch}")
        print(f"loss_seq: {loss_val}, loss_coord: {loss_coord_val}, loss_conf: {loss_conf_val}")
        if epoch >= len_pretraining:
            box_list = prediction_to_box_list(pred_sequence)
            dis_error = displacement_error(box_list, center_distance)
            writer.add_scalar('Val/loss_pred', loss_pred_val, (epoch+1) * epoch_len)
            writer.add_scalar('Val/dis_err', dis_error_val, (epoch+1) * epoch_len)
            print(f"loss_pred: {loss_pred_val}, dis_error: {dis_error_val}")
        print("###############")
        print("# VALIDATION END")
        print("###############")
        
        torch.save({'epoch': epoch, 'model_state_dict': opt.model.state_dict(),
                    'optimizer_state_dict': opt.optimizer_encoder.state_dict()},
                   opt.log_path+'/snapshot_encoder.tar')
        torch.save({'epoch': epoch, 'model_state_dict': opt.decoder.state_dict(),
                    'optimizer_state_dict': opt.optimizer_decoder.state_dict()},
                   opt.log_path+'/snapshot_decoder.tar')

    writer.close()

In [0]:
import warnings
warnings.filterwarnings('ignore')

train(opt)

NameError: ignored

In [29]:
#fire up tensorboard in the background
#todo make path first
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format('./log/')#opt.log_path)
)
get_ipython().system_raw('/content/ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://e04454aa.ngrok.io


In [0]:
opt = Opt()
opt.num_epoches = 100
opt.useCuda = True
opt.learning_rate = 3e-5
opt.decay = 1e-5
opt.batch_size = 1
opt.model = YoloFlowLSTM(opt.batch_size)
opt.decoder = ConvLSTM((opt.encoding_size, opt.encoding_size), 5 * 4, 1024, opt.batch_size)
opt.pre_trained_yolo_path = 'log/flow_1/snapshot_encoder.tar'
# load model
model_state_dict_yolo = torch.load(opt.pre_trained_yolo_path, map_location=opt.device())['model_state_dict']
opt.model.load_state_dict(model_state_dict_yolo, strict=True)
# all parameters are on the device the hidden states are not parameters
opt.model.to(opt.device())
opt.decoder.to(opt.device())
opt.yolo_loss = YoloLoss(opt.model.anchors, filter_fkt=filter_gt)
loss_params = {"grid_shape": (opt.encoding_size, opt.encoding_size),
               "image_shape": (opt.image_size, opt.image_size),
               "path_anchors": "dataset_utils/anchors/anchors5.txt"}
opt.pred_loss = NaiveLoss(loss_params)

In [0]:
def test_loss(pred, input, target):
    mask = target[:, :, :, :, 0] != 0
    #print(torch.sum(mask, dim=(0, 3)).detach().cpu().numpy())
    mask = torch.unsqueeze(mask.int(), 4)

    input = input[:, :, :, :, 1:]
    target = target[:, :, :, :, 1:]
    #print(f"dividend:  {torch.nonzero(mask).size(0)}")
    #print(torch.masked_select(pred, (pred.detach() * mask.detach() > 0).byte()))
    return torch.sum(mask.float() * torch.pow((target - input - pred), 2)) / torch.nonzero(mask).size(0)

def train_full(opt):
    # setup train and eval set
    torch.cuda.manual_seed(123)
    torch.manual_seed(123)
    
    # log stuff
    if os.path.isdir(opt.log_path):
        shutil.rmtree(opt.log_path)
    os.makedirs(opt.log_path)
    writer = SummaryWriter(opt.log_path)

    dataset = MotBBImageSequence('dataset_utils/Mot17_test_single.txt', use_only_first_video=False)
    train_data = Subset(dataset, range(0, dataset.valid_begin))
    valid_data = Subset(dataset, range(dataset.valid_begin, len(dataset)))
    train_loader = DataLoader(train_data, batch_size=opt.batch_size, shuffle=True, num_workers=opt.num_workers, drop_last=True)
    valid_loader = DataLoader(valid_data, batch_size=opt.batch_size, shuffle=False, num_workers=opt.num_workers, drop_last=True)

    opt.optimizer_encoder = torch.optim.Adam(opt.model.parameters(), lr=opt.learning_rate, betas=(opt.momentum, 0.999),
                                             weight_decay=opt.decay)
    opt.optimizer_decoder = torch.optim.Adam(opt.decoder.parameters(), lr=opt.learning_rate, betas=(opt.momentum, 0.999),
                                             weight_decay=opt.decay)

    obs_length = 10
    pred_length = 9
    
    epoch_len = len(train_loader)

    opt.model.train()
    opt.decoder.train()
    for epoch in range(opt.num_epoches):
        for img_num, (gt, b, img1, img2) in enumerate(train_loader):
            opt.model.lstm_part.reinit_lstm(opt.batch_size)
            opt.decoder.reset_hidden(opt.batch_size)
            opt.optimizer_encoder.zero_grad()
            opt.optimizer_decoder.zero_grad()
            
            seq_loss = 0
            seq_loss_coord, seq_loss_conf, seq_loss_pred = (0.0, 0.0, 0.0)
            pred_sequence = []
            for i in range(obs_length):  
                single_gt = b[:, i].to(opt.device())
                single_img2 = img2[:, i].to(opt.device())
                single_img2_normed = preprocess(img2[:, i]).to(opt.device())
                single_img1_normed = preprocess(img1[:, i]).to(opt.device())

                double_image = torch.cat((single_img1_normed, single_img2_normed), dim=1)

                logits = opt.model((double_image, single_img2))

                loss, loss_coord, loss_conf = opt.yolo_loss(logits, single_gt)
                seq_loss += loss
                seq_loss_conf += loss_conf.item()
                seq_loss_coord += loss_coord.item()

            prev_out = logits_to_box_params(logits.detach(), opt.model.anchors)
            # the box parameters are normalized to [0, 1]
            # at the moment [batch, anchors, ...., h* w]
            # rearrange tensor s.t [batch, h, w, anchors, ...]
            # ... == x, y, w, h, conf
            #print(f"origin mask {torch.sum(mask, dim=(0, 1))}")
            prev_out = prev_out.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                               .permute(0, 3, 4, 1, 2)
            # be carefull the position of the conf/id in the targets is 0 not 4
            # so we change it at this point to be consistent with the labels

            mask = (prev_out[:, :, :, :, 4] > 0.05).float()
            #print(f"origin mask {torch.sum(mask, dim=(0, 3))}")
            mask = mask.unsqueeze(-1)
            prev_out = torch.Tensor(np.roll(prev_out.cpu().numpy(), 1, axis=-1)).to(opt.device())

            opt.decoder.set_hidden(opt.model.lstm_part.hidden, opt.model.lstm_part.cell)
            for i in range(pred_length):
                # todo anchor box assignment of the target boxes does not use iou
                _, yolo_target = opt.pred_loss.to_yolo(input=gt[:, obs_length].numpy(), target=b[:, obs_length + i].numpy(), use_iou=True)
                yolo_target = torch.Tensor(yolo_target).to(opt.device())
                # target boxes are in [0, grid_h] -> normalize to 1
                # at this point i assume that the image is a square !!!
                yolo_target[:, :, :, :, 1:] = yolo_target[:, :, :, :, 1:] / opt.encoding_size
                #if i == 0:
                  #print(f"target mask {torch.sum((yolo_target[:, :, :, :, 0] > 0.0).float(), dim=(0, 3))}")
                prev_out = prev_out * mask  # todo maybe there is a better way
                #prev_out [batch, h, w, anchors, 4] 
                input = prev_out[:, :, :, :, 1:].contiguous()\
                                                .view(opt.batch_size, opt.encoding_size, opt.encoding_size, 
                                                      len(opt.model.anchors)*4)\
                                                .permute(0, 3, 1, 2)
                pred = opt.decoder(input)
                pred = pred.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                               .permute(0, 3, 4, 1, 2)

                seq_loss_pred += opt.pred_loss.forward(pred, prev_out, yolo_target)
                pred_sequence.append((prev_out.detach().cpu().numpy(),
                                      pred.detach().cpu().numpy(),
                                      (yolo_target).detach().cpu().numpy()))
                prev_out[:, :, :, :, 1:] = yolo_target[:, :, :, :, 1:]

            seq_loss += seq_loss_pred
            
            seq_loss.backward()
            opt.optimizer_encoder.step()
            opt.optimizer_decoder.step()
            
            seq_loss = seq_loss.item() - seq_loss_pred.item()
            
            writeLossToSummary(writer, 'Train', seq_loss/obs_length, 
                                                seq_loss_coord / obs_length,
                                                seq_loss_conf / obs_length, 
                                                epoch * epoch_len + img_num)

            print(f"epoch:{epoch} it: {img_num}")
            print(f"loss_seq: {seq_loss/obs_length}, loss_coord: {seq_loss_coord / obs_length}, loss_conf: {seq_loss_conf / obs_length}")

            seq_loss_pred = seq_loss_pred.item()
            box_list = prediction_to_box_list(pred_sequence)
            dis_error = displacement_error(box_list, center_distance)
            writer.add_scalar('Train/loss_pred', seq_loss_pred / pred_length, epoch * epoch_len + img_num)
            writer.add_scalar('Train/dis_err', dis_error / pred_length, epoch * epoch_len + img_num)
            print(f"loss_pred: {seq_loss_pred / pred_length}, dis_error: {dis_error / pred_length}")

            draw_pred_sequence(box_list, img2[0], pred_length, obs_length, name='train_img.png')
              
        ###############
        # VALIDATION
        ###############
        print("###############")
        print("# VALIDATION BEGIN")
        print("###############")
        opt.model.eval()
        opt.decoder.eval()
        valid_len = len(valid_loader)
        loss_val = 0
        loss_coord_val = 0
        loss_conf_val = 0
        loss_pred_val = 0
        dis_error_val = 0
        for img_num, (gt, b, img1, img2) in enumerate(valid_loader):
            opt.model.lstm_part.reinit_lstm(opt.batch_size)
            opt.decoder.reset_hidden(opt.batch_size)
            opt.optimizer_encoder.zero_grad()
            opt.optimizer_decoder.zero_grad()
            
            seq_loss = 0
            seq_loss_coord, seq_loss_conf, seq_loss_pred = (0.0, 0.0, 0.0)
            pred_sequence = []
            for i in range(obs_length):
                single_gt = b[:, i].to(opt.device())
                single_img2 = img2[:, i].to(opt.device())
                single_img2_normed = preprocess(img2[:, i]).to(opt.device())
                single_img1_normed = preprocess(img1[:, i]).to(opt.device())

                double_image = torch.cat((single_img1_normed, single_img2_normed), dim=1)
                with torch.no_grad():
                    logits = opt.model((double_image, single_img2))

                    loss, loss_coord, loss_conf = opt.yolo_loss(logits, single_gt)
                seq_loss += loss
                seq_loss_conf += loss_conf.item()
                seq_loss_coord += loss_coord.item()

            with torch.no_grad():
                prev_out = logits_to_box_params(logits.detach(), opt.model.anchors)
                # the box parameters are normalized to [0, 1]
                # at the moment [batch, anchors, ...., h* w]
                # rearrange tensor s.t [batch, h, w, anchors, ...]
                # ... == x, y, w, h, conf
                #print(f"origin mask {torch.sum(mask, dim=(0, 1))}")
                prev_out = prev_out.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                                   .permute(0, 3, 4, 1, 2)
                # be carefull the position of the conf/id in the targets is 0 not 4
                # so we change it at this point to be consistent with the labels

                #mask = (prev_out[:, :, :, :, 4] > 0.1).float()
                #print(f"origin mask {torch.sum(mask, dim=(0, 3))}")
                #mask = mask.unsqueeze(-1)
                prev_out = torch.Tensor(np.roll(prev_out.cpu().numpy(), 1, axis=-1)).to(opt.device())

                opt.decoder.set_hidden(opt.model.lstm_part.hidden, opt.model.lstm_part.cell)
                for i in range(pred_length):
                    # todo anchor box assignment of the target boxes does not use iou
                    _, yolo_target = opt.pred_loss.to_yolo(input=gt[:, obs_length].numpy(), target=b[:, obs_length + i].numpy(), use_iou=True)
                    yolo_target = torch.Tensor(yolo_target).to(opt.device())
                    # target boxes are in [0, grid_h] -> normalize to 1
                    # at this point i assume that the image is a square !!!
                    yolo_target[:, :, :, :, 1:] = yolo_target[:, :, :, :, 1:] / opt.encoding_size
                    #if i == 0:
                      #print(f"target mask {torch.sum((yolo_target[:, :, :, :, 0] > 0.0).float(), dim=(0, 3))}")
                    #prev_out = prev_out * mask  # todo maybe there is a better way
                    input = prev_out[:, :, :, :, 1:].contiguous()\
                                                    .view(opt.batch_size, len(opt.model.anchors)*4,
                                                          opt.encoding_size, opt.encoding_size)
                    pred = opt.decoder(input)
                    pred = pred.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                                   .permute(0, 3, 4, 1, 2)

                    seq_loss_pred += opt.pred_loss.forward(pred, prev_out, yolo_target)
                    pred_sequence.append((prev_out.detach().cpu().numpy(),
                                          pred.detach().cpu().numpy(),
                                          (yolo_target).detach().cpu().numpy()))
                    prev_out[:, :, :, :, 1:] += pred


                seq_loss_pred = seq_loss_pred.item()
                box_list = prediction_to_box_list(pred_sequence)
                dis_error = displacement_error(box_list, center_distance)
            
            loss_val += seq_loss.item() / valid_len /obs_length
            loss_coord_val += seq_loss_coord / valid_len /obs_length
            loss_conf_val += seq_loss_conf / valid_len /obs_length

            loss_pred_val += seq_loss_pred / valid_len  / pred_length
            dis_error_val += dis_error / valid_len  / pred_length
        draw_pred_sequence(box_list, img2[0], pred_length, obs_length, name='val_img.png')
        writeLossToSummary(writer, 'Val', loss_val, loss_coord_val,
                           loss_conf_val, (epoch+1) * epoch_len)

        print(f"epoch:{epoch}")
        print(f"loss_seq: {loss_val}, loss_coord: {loss_coord_val}, loss_conf: {loss_conf_val}")

        box_list = prediction_to_box_list(pred_sequence)
        dis_error = displacement_error(box_list, center_distance)
        writer.add_scalar('Val/loss_pred', loss_pred_val, (epoch+1) * epoch_len)
        writer.add_scalar('Val/dis_err', dis_error_val, (epoch+1) * epoch_len)
        print(f"loss_pred: {loss_pred_val}, dis_error: {dis_error_val}")
        print("###############")
        print("# VALIDATION END")
        print("###############")
        
        torch.save({'epoch': epoch, 'model_state_dict': opt.model.state_dict(),
                    'optimizer_state_dict': opt.optimizer_encoder.state_dict()},
                   opt.log_path+'/snapshot_encoder_fine.tar')
        torch.save({'epoch': epoch, 'model_state_dict': opt.decoder.state_dict(),
                    'optimizer_state_dict': opt.optimizer_decoder.state_dict()},
                   opt.log_path+'/snapshot_decoder.tar')

    writer.close()

In [0]:
import warnings
warnings.filterwarnings('ignore')

train_full(opt)

train real_decoder 1

In [0]:
opt = Opt()
opt.num_epoches = 100
opt.useCuda = True
opt.learning_rate = 3e-6
opt.decay = 1e-5
opt.batch_size = 1
opt.model = YoloFlowLSTM(opt.batch_size)
opt.decoder = ConvLSTM((opt.encoding_size, opt.encoding_size), 5 * 4, 1024, opt.batch_size)
opt.pre_trained_yolo_path = 'log/flow_1/snapshot_encoder.tar'
# load model
check_point_encoder = torch.load('log/flow_decoder_2/snapshot_encoder_fine.tar', map_location=opt.device())
check_point_decoder = torch.load('log/flow_decoder_2/snapshot_decoder.tar', map_location=opt.device())
model_state_dict_yolo = check_point_encoder['model_state_dict']
opt.model.load_state_dict(model_state_dict_yolo, strict=True)
opt.decoder.load_state_dict(check_point_decoder['model_state_dict'])
# all parameters are on the device the hidden states are not parameters
opt.model.to(opt.device())
opt.decoder.to(opt.device())
opt.yolo_loss = YoloLoss(opt.model.anchors, filter_fkt=filter_gt)
loss_params = {"grid_shape": (opt.encoding_size, opt.encoding_size),
               "image_shape": (opt.image_size, opt.image_size),
               "path_anchors": "dataset_utils/anchors/anchors5.txt"}
opt.pred_loss = NaiveLoss(loss_params)

opt.optimizer_encoder = torch.optim.Adam(opt.model.parameters(), lr=opt.learning_rate, betas=(opt.momentum, 0.999),
                                         weight_decay=opt.decay)
opt.optimizer_decoder = torch.optim.Adam(opt.decoder.parameters(), lr=opt.learning_rate, betas=(opt.momentum, 0.999),
                                         weight_decay=opt.decay)
opt.optimizer_encoder.load_state_dict(check_point_encoder['optimizer_state_dict'])
opt.optimizer_decoder.load_state_dict(check_point_decoder['optimizer_state_dict'])

In [0]:
def test_loss(pred, input, target):
    mask = target[:, :, :, :, 0] != 0
    #print(torch.sum(mask, dim=(0, 3)).detach().cpu().numpy())
    mask = torch.unsqueeze(mask.int(), 4)

    input = input[:, :, :, :, 1:]
    target = target[:, :, :, :, 1:]
    #print(f"dividend:  {torch.nonzero(mask).size(0)}")
    #print(torch.masked_select(pred, (pred.detach() * mask.detach() > 0).byte()))
    return torch.sum(mask.float() * torch.pow((target - input - pred), 2)) / torch.nonzero(mask).size(0)

def train_full(opt):
    # setup train and eval set
    torch.cuda.manual_seed(123)
    torch.manual_seed(123)
    
    # log stuff
    if os.path.isdir(opt.log_path):
        shutil.rmtree(opt.log_path)
    os.makedirs(opt.log_path)
    writer = SummaryWriter(opt.log_path)

    dataset = MotBBImageSequence('dataset_utils/Mot17_test_single.txt', use_only_first_video=False)
    train_data = Subset(dataset, range(0, dataset.valid_begin))
    valid_data = Subset(dataset, range(dataset.valid_begin, len(dataset)))
    train_loader = DataLoader(train_data, batch_size=opt.batch_size, shuffle=True, num_workers=opt.num_workers, drop_last=True)
    valid_loader = DataLoader(valid_data, batch_size=opt.batch_size, shuffle=False, num_workers=opt.num_workers, drop_last=True)
    
    obs_length = 10
    pred_length = 9
    
    epoch_len = len(train_loader)

    opt.model.train()
    opt.decoder.train()
    for epoch in range(opt.num_epoches):
        for img_num, (gt, b, img1, img2) in enumerate(train_loader):
            opt.model.lstm_part.reinit_lstm(opt.batch_size)
            opt.decoder.reset_hidden(opt.batch_size)
            opt.optimizer_encoder.zero_grad()
            opt.optimizer_decoder.zero_grad()
            
            seq_loss = 0
            seq_loss_coord, seq_loss_conf, seq_loss_pred = (0.0, 0.0, 0.0)
            pred_sequence = []
            for i in range(obs_length):  
                single_gt = b[:, i].to(opt.device())
                single_img2 = img2[:, i].to(opt.device())
                single_img2_normed = preprocess(img2[:, i]).to(opt.device())
                single_img1_normed = preprocess(img1[:, i]).to(opt.device())

                double_image = torch.cat((single_img1_normed, single_img2_normed), dim=1)

                logits = opt.model((double_image, single_img2))

                loss, loss_coord, loss_conf = opt.yolo_loss(logits, single_gt)
                seq_loss += loss
                seq_loss_conf += loss_conf.item()
                seq_loss_coord += loss_coord.item()

            prev_out = logits_to_box_params(logits.detach(), opt.model.anchors)
            # the box parameters are normalized to [0, 1]
            # at the moment [batch, anchors, ...., h* w]
            # rearrange tensor s.t [batch, h, w, anchors, ...]
            # ... == x, y, w, h, conf
            #print(f"origin mask {torch.sum(mask, dim=(0, 1))}")
            prev_out = prev_out.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                               .permute(0, 3, 4, 1, 2)
            # be carefull the position of the conf/id in the targets is 0 not 4
            # so we change it at this point to be consistent with the labels

            #mask = (prev_out[:, :, :, :, 4] > 0.1).float()
            #print(f"origin mask {torch.sum(mask, dim=(0, 3))}")
            #mask = mask.unsqueeze(-1)
            prev_out = torch.Tensor(np.roll(prev_out.cpu().numpy(), 1, axis=-1)).to(opt.device())

            opt.decoder.set_hidden(opt.model.lstm_part.hidden, opt.model.lstm_part.cell)
            for i in range(pred_length):
                # todo anchor box assignment of the target boxes does not use iou
                _, yolo_target = opt.pred_loss.to_yolo(input=gt[:, obs_length].numpy(), target=b[:, obs_length + i].numpy(), use_iou=True)
                yolo_target = torch.Tensor(yolo_target).to(opt.device())
                # target boxes are in [0, grid_h] -> normalize to 1
                # at this point i assume that the image is a square !!!
                yolo_target[:, :, :, :, 1:] = yolo_target[:, :, :, :, 1:] / opt.encoding_size
                #if i == 0:
                  #print(f"target mask {torch.sum((yolo_target[:, :, :, :, 0] > 0.0).float(), dim=(0, 3))}")
                #prev_out = prev_out * mask  # todo maybe there is a better way
                #prev_out [batch, h, w, anchors, 4] 
                input = prev_out[:, :, :, :, 1:].contiguous()\
                                                .view(opt.batch_size, opt.encoding_size, opt.encoding_size, 
                                                      len(opt.model.anchors)*4)\
                                                .permute(0, 3, 1, 2)
                pred = opt.decoder(input)
                pred = pred.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                               .permute(0, 3, 4, 1, 2)

                seq_loss_pred += opt.pred_loss.forward(pred, prev_out, yolo_target)
                pred_sequence.append((prev_out.detach().cpu().numpy(),
                                      pred.detach().cpu().numpy(),
                                      (yolo_target).detach().cpu().numpy()))
                prev_out[:, :, :, :, 1:] += pred #yolo_target[:, :, :, :, 1:]

            seq_loss += seq_loss_pred
            loss = seq_loss / 16
            loss.backward()
            if img_num % 16 == 15:
                opt.optimizer_encoder.step()
                opt.optimizer_decoder.step()

                opt.optimizer_encoder.zero_grad()
                opt.optimizer_decoder.zero_grad()
            
            seq_loss = seq_loss.item() - seq_loss_pred.item()
            
            writeLossToSummary(writer, 'Train', seq_loss/obs_length, 
                                                seq_loss_coord / obs_length,
                                                seq_loss_conf / obs_length, 
                                                epoch * epoch_len + img_num)

            print(f"epoch:{epoch} it: {img_num}")
            print(f"loss_seq: {seq_loss/obs_length}, loss_coord: {seq_loss_coord / obs_length}, loss_conf: {seq_loss_conf / obs_length}")

            seq_loss_pred = seq_loss_pred.item()
            box_list = prediction_to_box_list(pred_sequence)
            dis_error = displacement_error(box_list, center_distance)
            writer.add_scalar('Train/loss_pred', seq_loss_pred / pred_length, epoch * epoch_len + img_num)
            writer.add_scalar('Train/dis_err', dis_error, epoch * epoch_len + img_num)
            print(f"loss_pred: {seq_loss_pred / pred_length}, dis_error: {dis_error}")

            draw_pred_sequence(box_list, img2[0], pred_length, obs_length, name='train_img.png')
              
        ###############
        # VALIDATION
        ###############
        print("###############")
        print("# VALIDATION BEGIN")
        print("###############")
        opt.model.eval()
        opt.decoder.eval()
        valid_len = len(valid_loader)
        loss_val = 0
        loss_coord_val = 0
        loss_conf_val = 0
        loss_pred_val = 0
        dis_error_val = 0
        for img_num, (gt, b, img1, img2) in enumerate(valid_loader):
            opt.model.lstm_part.reinit_lstm(opt.batch_size)
            opt.decoder.reset_hidden(opt.batch_size)
            opt.optimizer_encoder.zero_grad()
            opt.optimizer_decoder.zero_grad()
            
            seq_loss = 0
            seq_loss_coord, seq_loss_conf, seq_loss_pred = (0.0, 0.0, 0.0)
            pred_sequence = []
            for i in range(obs_length):
                single_gt = b[:, i].to(opt.device())
                single_img2 = img2[:, i].to(opt.device())
                single_img2_normed = preprocess(img2[:, i]).to(opt.device())
                single_img1_normed = preprocess(img1[:, i]).to(opt.device())

                double_image = torch.cat((single_img1_normed, single_img2_normed), dim=1)
                with torch.no_grad():
                    logits = opt.model((double_image, single_img2))

                    loss, loss_coord, loss_conf = opt.yolo_loss(logits, single_gt)
                seq_loss += loss
                seq_loss_conf += loss_conf.item()
                seq_loss_coord += loss_coord.item()

            with torch.no_grad():
                prev_out = logits_to_box_params(logits.detach(), opt.model.anchors)
                # the box parameters are normalized to [0, 1]
                # at the moment [batch, anchors, ...., h* w]
                # rearrange tensor s.t [batch, h, w, anchors, ...]
                # ... == x, y, w, h, conf
                #print(f"origin mask {torch.sum(mask, dim=(0, 1))}")
                prev_out = prev_out.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                                   .permute(0, 3, 4, 1, 2)
                # be carefull the position of the conf/id in the targets is 0 not 4
                # so we change it at this point to be consistent with the labels

                #mask = (prev_out[:, :, :, :, 4] > 0.25).float()
                #print(f"origin mask {torch.sum(mask, dim=(0, 3))}")
                #mask = mask.unsqueeze(-1)
                prev_out = torch.Tensor(np.roll(prev_out.cpu().numpy(), 1, axis=-1)).to(opt.device())

                opt.decoder.set_hidden(opt.model.lstm_part.hidden, opt.model.lstm_part.cell)
                for i in range(pred_length):
                    # todo anchor box assignment of the target boxes does not use iou
                    _, yolo_target = opt.pred_loss.to_yolo(input=gt[:, obs_length].numpy(), target=b[:, obs_length + i].numpy(), use_iou=True)
                    yolo_target = torch.Tensor(yolo_target).to(opt.device())
                    # target boxes are in [0, grid_h] -> normalize to 1
                    # at this point i assume that the image is a square !!!
                    yolo_target[:, :, :, :, 1:] = yolo_target[:, :, :, :, 1:] / opt.encoding_size
                    #if i == 0:
                      #print(f"target mask {torch.sum((yolo_target[:, :, :, :, 0] > 0.0).float(), dim=(0, 3))}")
                    #prev_out = prev_out * mask  # todo maybe there is a better way
                    input = prev_out[:, :, :, :, 1:].contiguous()\
                                                    .view(opt.batch_size, opt.encoding_size, opt.encoding_size, 
                                                          len(opt.model.anchors)*4)\
                                                    .permute(0, 3, 1, 2)
                    pred = opt.decoder(input)
                    pred = pred.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                                   .permute(0, 3, 4, 1, 2)

                    seq_loss_pred += opt.pred_loss.forward(pred, prev_out, yolo_target)
                    pred_sequence.append((prev_out.detach().cpu().numpy(),
                                          pred.detach().cpu().numpy(),
                                          (yolo_target).detach().cpu().numpy()))
                    prev_out[:, :, :, :, 1:] += pred


                seq_loss_pred = seq_loss_pred.item()
                box_list = prediction_to_box_list(pred_sequence)
                dis_error = displacement_error(box_list, center_distance)
            
            loss_val += seq_loss.item() /valid_len /obs_length
            loss_coord_val += seq_loss_coord /valid_len /obs_length
            loss_conf_val += seq_loss_conf /valid_len /obs_length

            loss_pred_val += seq_loss_pred /valid_len  / pred_length
            dis_error_val += dis_error /valid_len
        draw_pred_sequence(box_list, img2[0], pred_length, obs_length, name=f'val_img.png')
        writeLossToSummary(writer, 'Val', loss_val, loss_coord_val,
                           loss_conf_val, (epoch+1) * epoch_len)

        print(f"epoch:{epoch}")
        print(f"loss_seq: {loss_val}, loss_coord: {loss_coord_val}, loss_conf: {loss_conf_val}")

        box_list = prediction_to_box_list(pred_sequence)
        dis_error = displacement_error(box_list, center_distance)
        writer.add_scalar('Val/loss_pred', loss_pred_val, (epoch+1) * epoch_len)
        writer.add_scalar('Val/dis_err', dis_error_val, (epoch+1) * epoch_len)
        print(f"loss_pred: {loss_pred_val}, dis_error: {dis_error_val}")
        print("###############")
        print("# VALIDATION END")
        print("###############")
        
        torch.save({'epoch': epoch, 'model_state_dict': opt.model.state_dict(),
                    'optimizer_state_dict': opt.optimizer_encoder.state_dict()},
                   opt.log_path+'/snapshot_encoder_fine.tar')
        torch.save({'epoch': epoch, 'model_state_dict': opt.decoder.state_dict(),
                    'optimizer_state_dict': opt.optimizer_decoder.state_dict()},
                   opt.log_path+'/snapshot_decoder.tar')

    writer.close()

In [0]:
import warnings
warnings.filterwarnings('ignore')

train_full(opt)

Traininert flow_decoder_real weiter (encoder was not saved and decoder only an old state -> change for future)


In [0]:
opt = Opt()
opt.num_epoches = 100
opt.useCuda = True
opt.learning_rate = 3e-5
opt.decay = 1e-5
opt.batch_size = 1
opt.model = YoloFlowLSTM(opt.batch_size)
opt.decoder = ConvLSTM((opt.encoding_size, opt.encoding_size), 5 * 4, 1024, opt.batch_size)
opt.pre_trained_yolo_path = 'log/flow_1/snapshot_encoder.tar'
# load model
check_point_encoder = torch.load('log/flow_1/snapshot_encoder.tar', map_location=opt.device())
check_point_decoder = torch.load('log/flow_real_2_decoder_1/snapshot_decoder.tar', map_location=opt.device())
model_state_dict_yolo = check_point_encoder['model_state_dict']
opt.model.load_state_dict(model_state_dict_yolo, strict=True)
opt.decoder.load_state_dict(check_point_decoder['model_state_dict'])
# all parameters are on the device the hidden states are not parameters
opt.model.to(opt.device())
opt.decoder.to(opt.device())
opt.yolo_loss = YoloLoss(opt.model.anchors, filter_fkt=filter_gt)
loss_params = {"grid_shape": (opt.encoding_size, opt.encoding_size),
               "image_shape": (opt.image_size, opt.image_size),
               "path_anchors": "dataset_utils/anchors/anchors5.txt"}
opt.pred_loss = NaiveLoss(loss_params)

opt.optimizer_encoder = torch.optim.Adam(opt.model.parameters(), lr=opt.learning_rate, betas=(opt.momentum, 0.999),
                                         weight_decay=opt.decay)
opt.optimizer_decoder = torch.optim.Adam(opt.decoder.parameters(), lr=opt.learning_rate, betas=(opt.momentum, 0.999),
                                         weight_decay=opt.decay)
#opt.optimizer_encoder.load_state_dict(check_point_encoder['optimizer_state_dict'])
opt.optimizer_decoder.load_state_dict(check_point_decoder['optimizer_state_dict'])

In [0]:

def test_loss(pred, input, target):
    mask = target[:, :, :, :, 0] != 0
    #print(torch.sum(mask, dim=(0, 3)).detach().cpu().numpy())
    mask = torch.unsqueeze(mask.int(), 4)

    input = input[:, :, :, :, 1:]
    target = target[:, :, :, :, 1:]
    #print(f"dividend:  {torch.nonzero(mask).size(0)}")
    #print(torch.masked_select(pred, (pred.detach() * mask.detach() > 0).byte()))
    return torch.sum(mask.float() * torch.pow((target - input - pred), 2)) / torch.nonzero(mask).size(0)

def train_full_2(opt):
    # setup train and eval set
    torch.cuda.manual_seed(123)
    torch.manual_seed(123)
    
    # log stuff
    if os.path.isdir(opt.log_path):
        shutil.rmtree(opt.log_path)
    os.makedirs(opt.log_path)
    writer = SummaryWriter(opt.log_path)

    dataset = MotBBImageSequence('dataset_utils/Mot17_test_single.txt', use_only_first_video=False)
    train_data = Subset(dataset, range(0, dataset.valid_begin))
    valid_data = Subset(dataset, range(dataset.valid_begin, len(dataset)))
    train_loader = DataLoader(train_data, batch_size=opt.batch_size, shuffle=True, num_workers=opt.num_workers, drop_last=True)
    valid_loader = DataLoader(valid_data, batch_size=opt.batch_size, shuffle=False, num_workers=opt.num_workers, drop_last=True)

    
    obs_length = 10
    pred_length = 9
    
    epoch_len = len(train_loader)

    opt.model.train()
    opt.decoder.train()
    for epoch in range(opt.num_epoches):
        for img_num, (gt, b, img1, img2) in enumerate(train_loader):
            opt.model.lstm_part.reinit_lstm(opt.batch_size)
            opt.decoder.reset_hidden(opt.batch_size)
            opt.optimizer_encoder.zero_grad()
            opt.optimizer_decoder.zero_grad()
            
            seq_loss = 0
            seq_loss_coord, seq_loss_conf, seq_loss_pred = (0.0, 0.0, 0.0)
            pred_sequence = []
            for i in range(obs_length):  
                single_gt = b[:, i].to(opt.device())
                single_img2 = img2[:, i].to(opt.device())
                single_img2_normed = preprocess(img2[:, i]).to(opt.device())
                single_img1_normed = preprocess(img1[:, i]).to(opt.device())

                double_image = torch.cat((single_img1_normed, single_img2_normed), dim=1)

                logits = opt.model((double_image, single_img2))

                loss, loss_coord, loss_conf = opt.yolo_loss(logits, single_gt)
                seq_loss += loss
                seq_loss_conf += loss_conf.item()
                seq_loss_coord += loss_coord.item()

            prev_out = logits_to_box_params(logits.detach(), opt.model.anchors)
            # the box parameters are normalized to [0, 1]
            # at the moment [batch, anchors, ...., h* w]
            # rearrange tensor s.t [batch, h, w, anchors, ...]
            # ... == x, y, w, h, conf
            #print(f"origin mask {torch.sum(mask, dim=(0, 1))}")
            prev_out = prev_out.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                               .permute(0, 3, 4, 1, 2)
            # be carefull the position of the conf/id in the targets is 0 not 4
            # so we change it at this point to be consistent with the labels

            #mask = (prev_out[:, :, :, :, 4] > 0.1).float()
            #print(f"origin mask {torch.sum(mask, dim=(0, 3))}")
            #mask = mask.unsqueeze(-1)
            prev_out = torch.Tensor(np.roll(prev_out.cpu().numpy(), 1, axis=-1)).to(opt.device())

            opt.decoder.set_hidden(opt.model.lstm_part.hidden, opt.model.lstm_part.cell)
            for i in range(pred_length):
                # todo anchor box assignment of the target boxes does not use iou
                _, yolo_target = opt.pred_loss.to_yolo(input=gt[:, obs_length].numpy(), target=b[:, obs_length + i].numpy(), use_iou=True)
                yolo_target = torch.Tensor(yolo_target).to(opt.device())
                # target boxes are in [0, grid_h] -> normalize to 1
                # at this point i assume that the image is a square !!!
                yolo_target[:, :, :, :, 1:] = yolo_target[:, :, :, :, 1:] / opt.encoding_size
                #if i == 0:
                  #print(f"target mask {torch.sum((yolo_target[:, :, :, :, 0] > 0.0).float(), dim=(0, 3))}")
                #prev_out = prev_out * mask  # todo maybe there is a better way
                #prev_out [batch, h, w, anchors, 4] 
                input = prev_out[:, :, :, :, 1:].contiguous()\
                                                .view(opt.batch_size, opt.encoding_size, opt.encoding_size, 
                                                      len(opt.model.anchors)*4)\
                                                .permute(0, 3, 1, 2)
                pred = opt.decoder(input)
                pred = pred.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                               .permute(0, 3, 4, 1, 2)

                seq_loss_pred += opt.pred_loss.forward(pred, prev_out, yolo_target)
                pred_sequence.append((prev_out.detach().cpu().numpy(),
                                      pred.detach().cpu().numpy(),
                                      (yolo_target).detach().cpu().numpy()))
                prev_out[:, :, :, :, 1:] += pred #yolo_target[:, :, :, :, 1:]

            seq_loss += seq_loss_pred
            loss = seq_loss / 16
            loss.backward()
            if img_num % 16 == 15:
                opt.optimizer_encoder.step()
                opt.optimizer_decoder.step()

                opt.optimizer_encoder.zero_grad()
                opt.optimizer_decoder.zero_grad()
            
            seq_loss = seq_loss.item() - seq_loss_pred.item()
            
            writeLossToSummary(writer, 'Train', seq_loss/obs_length, 
                                                seq_loss_coord / obs_length,
                                                seq_loss_conf / obs_length, 
                                                epoch * epoch_len + img_num)

            print(f"epoch:{epoch} it: {img_num}")
            print(f"loss_seq: {seq_loss/obs_length}, loss_coord: {seq_loss_coord / obs_length}, loss_conf: {seq_loss_conf / obs_length}")

            seq_loss_pred = seq_loss_pred.item()
            box_list = prediction_to_box_list(pred_sequence)
            dis_error = displacement_error(box_list, center_distance)
            writer.add_scalar('Train/loss_pred', seq_loss_pred / pred_length, epoch * epoch_len + img_num)
            writer.add_scalar('Train/dis_err', dis_error, epoch * epoch_len + img_num)
            print(f"loss_pred: {seq_loss_pred / pred_length}, dis_error: {dis_error}")

            draw_pred_sequence(box_list, img2[0], pred_length, obs_length, name='train_img.png')
              
        ###############
        # VALIDATION
        ###############
        print("###############")
        print("# VALIDATION BEGIN")
        print("###############")
        opt.model.eval()
        opt.decoder.eval()
        valid_len = len(valid_loader)
        loss_val = 0
        loss_coord_val = 0
        loss_conf_val = 0
        loss_pred_val = 0
        dis_error_val = 0
        for img_num, (gt, b, img1, img2) in enumerate(valid_loader):
            opt.model.lstm_part.reinit_lstm(opt.batch_size)
            opt.decoder.reset_hidden(opt.batch_size)
            opt.optimizer_encoder.zero_grad()
            opt.optimizer_decoder.zero_grad()
            
            seq_loss = 0
            seq_loss_coord, seq_loss_conf, seq_loss_pred = (0.0, 0.0, 0.0)
            pred_sequence = []
            for i in range(obs_length):
                single_gt = b[:, i].to(opt.device())
                single_img2 = img2[:, i].to(opt.device())
                single_img2_normed = preprocess(img2[:, i]).to(opt.device())
                single_img1_normed = preprocess(img1[:, i]).to(opt.device())

                double_image = torch.cat((single_img1_normed, single_img2_normed), dim=1)
                with torch.no_grad():
                    logits = opt.model((double_image, single_img2))

                    loss, loss_coord, loss_conf = opt.yolo_loss(logits, single_gt)
                seq_loss += loss
                seq_loss_conf += loss_conf.item()
                seq_loss_coord += loss_coord.item()

            with torch.no_grad():
                prev_out = logits_to_box_params(logits.detach(), opt.model.anchors)
                # the box parameters are normalized to [0, 1]
                # at the moment [batch, anchors, ...., h* w]
                # rearrange tensor s.t [batch, h, w, anchors, ...]
                # ... == x, y, w, h, conf
                #print(f"origin mask {torch.sum(mask, dim=(0, 1))}")
                prev_out = prev_out.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                                   .permute(0, 3, 4, 1, 2)
                # be carefull the position of the conf/id in the targets is 0 not 4
                # so we change it at this point to be consistent with the labels

                #mask = (prev_out[:, :, :, :, 4] > 0.25).float()
                #print(f"origin mask {torch.sum(mask, dim=(0, 3))}")
                #mask = mask.unsqueeze(-1)
                prev_out = torch.Tensor(np.roll(prev_out.cpu().numpy(), 1, axis=-1)).to(opt.device())

                opt.decoder.set_hidden(opt.model.lstm_part.hidden, opt.model.lstm_part.cell)
                for i in range(pred_length):
                    # todo anchor box assignment of the target boxes does not use iou
                    _, yolo_target = opt.pred_loss.to_yolo(input=gt[:, obs_length].numpy(), target=b[:, obs_length + i].numpy(), use_iou=True)
                    yolo_target = torch.Tensor(yolo_target).to(opt.device())
                    # target boxes are in [0, grid_h] -> normalize to 1
                    # at this point i assume that the image is a square !!!
                    yolo_target[:, :, :, :, 1:] = yolo_target[:, :, :, :, 1:] / opt.encoding_size
                    #if i == 0:
                      #print(f"target mask {torch.sum((yolo_target[:, :, :, :, 0] > 0.0).float(), dim=(0, 3))}")
                    #prev_out = prev_out * mask  # todo maybe there is a better way
                    input = prev_out[:, :, :, :, 1:].contiguous()\
                                                    .view(opt.batch_size, opt.encoding_size, opt.encoding_size, 
                                                          len(opt.model.anchors)*4)\
                                                    .permute(0, 3, 1, 2)
                    pred = opt.decoder(input)
                    pred = pred.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                                   .permute(0, 3, 4, 1, 2)

                    seq_loss_pred += opt.pred_loss.forward(pred, prev_out, yolo_target)
                    pred_sequence.append((prev_out.detach().cpu().numpy(),
                                          pred.detach().cpu().numpy(),
                                          (yolo_target).detach().cpu().numpy()))
                    prev_out[:, :, :, :, 1:] += pred


                seq_loss_pred = seq_loss_pred.item()
                box_list = prediction_to_box_list(pred_sequence)
                dis_error = displacement_error(box_list, center_distance)
            
            loss_val += seq_loss.item() /valid_len /obs_length
            loss_coord_val += seq_loss_coord /valid_len /obs_length
            loss_conf_val += seq_loss_conf /valid_len /obs_length

            loss_pred_val += seq_loss_pred /valid_len  / pred_length
            dis_error_val += dis_error /valid_len
        draw_pred_sequence(box_list, img2[0], pred_length, obs_length, name=f'val_img.png')
        writeLossToSummary(writer, 'Val', loss_val, loss_coord_val,
                           loss_conf_val, (epoch+1) * epoch_len)

        print(f"epoch:{epoch}")
        print(f"loss_seq: {loss_val}, loss_coord: {loss_coord_val}, loss_conf: {loss_conf_val}")

        box_list = prediction_to_box_list(pred_sequence)
        dis_error = displacement_error(box_list, center_distance)
        writer.add_scalar('Val/loss_pred', loss_pred_val, (epoch+1) * epoch_len)
        writer.add_scalar('Val/dis_err', dis_error_val, (epoch+1) * epoch_len)
        print(f"loss_pred: {loss_pred_val}, dis_error: {dis_error_val}")
        print("###############")
        print("# VALIDATION END")
        print("###############")
        
        torch.save({'epoch': epoch, 'model_state_dict': opt.model.state_dict(),
                    'optimizer_state_dict': opt.optimizer_encoder.state_dict()},
                   opt.log_path+'/snapshot_encoder_fine.tar')
        torch.save({'epoch': epoch, 'model_state_dict': opt.decoder.state_dict(),
                    'optimizer_state_dict': opt.optimizer_decoder.state_dict()},
                   opt.log_path+'/snapshot_decoder.tar')

    writer.close()

In [0]:
import warnings
warnings.filterwarnings('ignore')

train_full_2(opt)

Train from base with random fake or real labels ( first epoch are real labels)

In [0]:
opt = Opt()
opt.num_epoches = 100
opt.useCuda = True
opt.learning_rate = 3e-5
opt.decay = 1e-5
opt.batch_size = 1
opt.model = YoloFlowLSTM(opt.batch_size)
opt.decoder = ConvLSTM((opt.encoding_size, opt.encoding_size), 5 * 4, 1024, opt.batch_size)
opt.pre_trained_yolo_path = 'log/flow_1/snapshot_encoder.tar'
# load model
check_point_encoder = torch.load('log/flow_1/snapshot_encoder.tar', map_location=opt.device())
model_state_dict_yolo = check_point_encoder['model_state_dict']
opt.model.load_state_dict(model_state_dict_yolo, strict=True)
# all parameters are on the device the hidden states are not parameters
opt.model.to(opt.device())
opt.decoder.to(opt.device())
opt.yolo_loss = YoloLoss(opt.model.anchors, filter_fkt=filter_gt)
loss_params = {"grid_shape": (opt.encoding_size, opt.encoding_size),
               "image_shape": (opt.image_size, opt.image_size),
               "path_anchors": "dataset_utils/anchors/anchors5.txt"}
opt.pred_loss = NaiveLoss(loss_params)

opt.optimizer_encoder = torch.optim.Adam(opt.model.parameters(), lr=opt.learning_rate, betas=(opt.momentum, 0.999),
                                         weight_decay=opt.decay)
opt.optimizer_decoder = torch.optim.Adam(opt.decoder.parameters(), lr=opt.learning_rate, betas=(opt.momentum, 0.999),
                                         weight_decay=opt.decay)

In [0]:
import random

def filter_gt_batch(gt):

    # used to filter only existing bb for loss
    t = gt[gt[:, :, 0] != 0.0]
   
    return t[:, 1:]

def train_full_3(opt):
    # setup train and eval set
    #torch.cuda.manual_seed(123)
    #torch.manual_seed(123)
    
    # log stuff
    if os.path.isdir(opt.log_path):
        shutil.rmtree(opt.log_path)
    os.makedirs(opt.log_path)
    writer = SummaryWriter(opt.log_path)

    dataset = MotBBImageSequence('dataset_utils/Mot17_test_single.txt', use_only_first_video=False)
    train_data = Subset(dataset, range(0, dataset.valid_begin))
    valid_data = Subset(dataset, range(dataset.valid_begin, len(dataset)))
    train_loader = DataLoader(train_data, batch_size=opt.batch_size, shuffle=True, num_workers=opt.num_workers, drop_last=True)
    valid_loader = DataLoader(valid_data, batch_size=opt.batch_size, shuffle=True, num_workers=opt.num_workers, drop_last=True)
    
    obs_length = 10
    pred_length = 9
    
    epoch_len = len(train_loader)

    opt.model.train()
    opt.decoder.train()
    
    for epoch in range(opt.num_epoches):
        for img_num, (gt, b, img1, img2) in enumerate(train_loader):
            opt.model.lstm_part.reinit_lstm(opt.batch_size)
            opt.decoder.reset_hidden(opt.batch_size)
            
            seq_loss = 0
            seq_ap = 0
            seq_loss_coord, seq_loss_conf, seq_loss_pred = (0.0, 0.0, 0.0)
            pred_sequence = []
            for i in range(obs_length):  
                single_gt = b[:, i].to(opt.device())
                single_img2 = img2[:, i].to(opt.device())
                single_img2_normed = preprocess(img2[:, i]).to(opt.device())
                single_img1_normed = preprocess(img1[:, i]).to(opt.device())

                double_image = torch.cat((single_img1_normed, single_img2_normed), dim=1)

                logits = opt.model((double_image, single_img2))

                loss, loss_coord, loss_conf = opt.yolo_loss(logits, single_gt)
                seq_ap += get_ap(logits, filter_gt_batch(single_gt), opt.image_size, opt.image_size, opt.model.anchors)
                seq_loss += loss
                seq_loss_conf += loss_conf.item()
                seq_loss_coord += loss_coord.item()

            prev_out = logits_to_box_params(logits.detach(), opt.model.anchors)
            # the box parameters are normalized to [0, 1]
            # at the moment [batch, anchors, ...., h* w]
            # rearrange tensor s.t [batch, h, w, anchors, ...]
            # ... == x, y, w, h, conf
            #print(f"origin mask {torch.sum(mask, dim=(0, 1))}")
            prev_out = prev_out.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                               .permute(0, 3, 4, 1, 2)
            # be carefull the position of the conf/id in the targets is 0 not 4
            # so we change it at this point to be consistent with the labels

            #mask = (prev_out[:, :, :, :, 4] > 0.1).float()
            #print(f"origin mask {torch.sum(mask, dim=(0, 3))}")
            #mask = mask.unsqueeze(-1)
            prev_out = torch.Tensor(np.roll(prev_out.cpu().numpy(), 1, axis=-1)).to(opt.device())

            opt.decoder.set_hidden(opt.model.lstm_part.hidden, opt.model.lstm_part.cell)
            
            teacher_forcing_ratio = 0.5 if epoch >= 1 else 1
            use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

            for i in range(pred_length):
                # todo anchor box assignment of the target boxes does not use iou
                _, yolo_target = opt.pred_loss.to_yolo(input=gt[:, obs_length].numpy(), target=b[:, obs_length + i].numpy(), use_iou=True)
                yolo_target = torch.Tensor(yolo_target).to(opt.device())
                # target boxes are in [0, grid_h] -> normalize to 1
                # at this point i assume that the image is a square !!!
                yolo_target[:, :, :, :, 1:] = yolo_target[:, :, :, :, 1:] / opt.encoding_size
                #if i == 0:
                  #print(f"target mask {torch.sum((yolo_target[:, :, :, :, 0] > 0.0).float(), dim=(0, 3))}")
                #prev_out = prev_out * mask  # todo maybe there is a better way
                #prev_out [batch, h, w, anchors, 4] 
                input = prev_out[:, :, :, :, 1:].contiguous()\
                                                .view(opt.batch_size, opt.encoding_size, opt.encoding_size, 
                                                      len(opt.model.anchors)*4)\
                                                .permute(0, 3, 1, 2)
                pred = opt.decoder(input)
                pred = pred.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                               .permute(0, 3, 4, 1, 2)

                seq_loss_pred += opt.pred_loss.forward(pred, prev_out, yolo_target)
                pred_sequence.append((prev_out.detach().cpu().numpy(),
                                      pred.detach().cpu().numpy(),
                                      (yolo_target).detach().cpu().numpy()))
                if not use_teacher_forcing:
                    prev_out[:, :, :, :, 1:] += pred
                else:
                    prev_out[:, :, :, :, 1:] = yolo_target[:, :, :, :, 1:]

            seq_loss += seq_loss_pred
            loss = seq_loss / 16
            loss.backward()
            if img_num % 16 == 15:
                opt.optimizer_encoder.step()
                opt.optimizer_decoder.step()

                opt.optimizer_encoder.zero_grad()
                opt.optimizer_decoder.zero_grad()
            
            seq_loss = seq_loss.item() - seq_loss_pred.item()
            
            writeLossToSummary(writer, 'Train', seq_loss/obs_length, 
                                                seq_loss_coord / obs_length,
                                                seq_loss_conf / obs_length, 
                                                epoch * epoch_len + img_num)

            print(f"epoch:{epoch} it: {img_num}")
            print(f"loss_seq: {seq_loss/obs_length}, loss_coord: {seq_loss_coord / obs_length}, loss_conf: {seq_loss_conf / obs_length}, mAP: {seq_ap/obs_length}")

            seq_loss_pred = seq_loss_pred.item()
            box_list = prediction_to_box_list(pred_sequence)
            dis_error = displacement_error(box_list, center_distance)
            writer.add_scalar('Train/loss_pred', seq_loss_pred / pred_length, epoch * epoch_len + img_num)
            writer.add_scalar('Train/dis_err', dis_error, epoch * epoch_len + img_num)
            writer.add_scalar('Train/AP', seq_ap/obs_length, epoch * epoch_len + img_num)
            print(f"loss_pred: {seq_loss_pred / pred_length}, dis_error: {dis_error}")

            draw_pred_sequence(box_list, img2[0], pred_length, obs_length, name='train_img.png')
              
        ###############
        # VALIDATION
        ###############
        print("###############")
        print("# VALIDATION BEGIN")
        print("###############")
        opt.model.eval()
        opt.decoder.eval()
        valid_len = len(valid_loader)
        loss_val = 0
        loss_ap = 0
        loss_coord_val = 0
        loss_conf_val = 0
        loss_pred_val = 0
        dis_error_val = 0
        for img_num, (gt, b, img1, img2) in enumerate(valid_loader):
            opt.model.lstm_part.reinit_lstm(opt.batch_size)
            opt.decoder.reset_hidden(opt.batch_size)
            opt.optimizer_encoder.zero_grad()
            opt.optimizer_decoder.zero_grad()
            
            seq_loss = 0
            seq_loss_coord, seq_loss_conf, seq_loss_pred = (0.0, 0.0, 0.0)
            seq_ap = 0
            pred_sequence = []
            for i in range(obs_length):
                single_gt = b[:, i].to(opt.device())
                single_img2 = img2[:, i].to(opt.device())
                single_img2_normed = preprocess(img2[:, i]).to(opt.device())
                single_img1_normed = preprocess(img1[:, i]).to(opt.device())

                double_image = torch.cat((single_img1_normed, single_img2_normed), dim=1)
                with torch.no_grad():
                    logits = opt.model((double_image, single_img2))

                    loss, loss_coord, loss_conf = opt.yolo_loss(logits, single_gt)
                    
                seq_ap += get_ap(logits.detach(), filter_gt_batch(single_gt), opt.image_size, opt.image_size, opt.model.anchors)
                seq_loss += loss
                seq_loss_conf += loss_conf.item()
                seq_loss_coord += loss_coord.item()

            with torch.no_grad():
                prev_out = logits_to_box_params(logits.detach(), opt.model.anchors)
                # the box parameters are normalized to [0, 1]
                # at the moment [batch, anchors, ...., h* w]
                # rearrange tensor s.t [batch, h, w, anchors, ...]
                # ... == x, y, w, h, conf
                #print(f"origin mask {torch.sum(mask, dim=(0, 1))}")
                prev_out = prev_out.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                                   .permute(0, 3, 4, 1, 2)
                # be carefull the position of the conf/id in the targets is 0 not 4
                # so we change it at this point to be consistent with the labels

                #mask = (prev_out[:, :, :, :, 4] > 0.25).float()
                #print(f"origin mask {torch.sum(mask, dim=(0, 3))}")
                #mask = mask.unsqueeze(-1)
                prev_out = torch.Tensor(np.roll(prev_out.cpu().numpy(), 1, axis=-1)).to(opt.device())

                opt.decoder.set_hidden(opt.model.lstm_part.hidden, opt.model.lstm_part.cell)
                for i in range(pred_length):
                    # todo anchor box assignment of the target boxes does not use iou
                    _, yolo_target = opt.pred_loss.to_yolo(input=gt[:, obs_length].numpy(), target=b[:, obs_length + i].numpy(), use_iou=True)
                    yolo_target = torch.Tensor(yolo_target).to(opt.device())
                    # target boxes are in [0, grid_h] -> normalize to 1
                    # at this point i assume that the image is a square !!!
                    yolo_target[:, :, :, :, 1:] = yolo_target[:, :, :, :, 1:] / opt.encoding_size
                    #if i == 0:
                      #print(f"target mask {torch.sum((yolo_target[:, :, :, :, 0] > 0.0).float(), dim=(0, 3))}")
                    #prev_out = prev_out * mask  # todo maybe there is a better way
                    input = prev_out[:, :, :, :, 1:].contiguous()\
                                                    .view(opt.batch_size, opt.encoding_size, opt.encoding_size, 
                                                          len(opt.model.anchors)*4)\
                                                    .permute(0, 3, 1, 2)
                    pred = opt.decoder(input)
                    pred = pred.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                                   .permute(0, 3, 4, 1, 2)

                    seq_loss_pred += opt.pred_loss.forward(pred, prev_out, yolo_target)
                    pred_sequence.append((prev_out.detach().cpu().numpy(),
                                          pred.detach().cpu().numpy(),
                                          (yolo_target).detach().cpu().numpy()))
                    prev_out[:, :, :, :, 1:] += pred


                seq_loss_pred = seq_loss_pred.item()
                box_list = prediction_to_box_list(pred_sequence)
                dis_error = displacement_error(box_list, center_distance)
            
            loss_val += seq_loss.item() /valid_len /obs_length
            loss_coord_val += seq_loss_coord /valid_len /obs_length
            loss_conf_val += seq_loss_conf /valid_len /obs_length
            loss_ap += seq_ap / obs_length / valid_len

            loss_pred_val += seq_loss_pred /valid_len  / pred_length
            dis_error_val += dis_error /valid_len
        draw_pred_sequence(box_list, img2[0], pred_length, obs_length, name=f'val_img.png')
        writeLossToSummary(writer, 'Val', loss_val, loss_coord_val,
                           loss_conf_val, (epoch+1) * epoch_len)

        print(f"epoch:{epoch}")
        print(f"loss_seq: {loss_val}, loss_coord: {loss_coord_val}, loss_conf: {loss_conf_val}, mAP: {loss_ap}")

        box_list = prediction_to_box_list(pred_sequence)
        dis_error = displacement_error(box_list, center_distance)
        writer.add_scalar('Val/loss_pred', loss_pred_val, (epoch+1) * epoch_len)
        writer.add_scalar('Val/dis_err', dis_error_val, (epoch+1) * epoch_len)
        writer.add_scalar('Val/mAP', loss_ap, (epoch+1) * epoch_len)

        print(f"loss_pred: {loss_pred_val}, dis_error: {dis_error_val}")
        print("###############")
        print("# VALIDATION END")
        print("###############")
        
        torch.save({'epoch': epoch, 'model_state_dict': opt.model.state_dict(),
                    'optimizer_state_dict': opt.optimizer_encoder.state_dict()},
                   opt.log_path+'/snapshot_encoder_fine.tar')
        torch.save({'epoch': epoch, 'model_state_dict': opt.decoder.state_dict(),
                    'optimizer_state_dict': opt.optimizer_decoder.state_dict()},
                   opt.log_path+'/snapshot_decoder.tar')

    writer.close()

In [0]:
import warnings
warnings.filterwarnings('ignore')

train_full_3(opt)

finetune flow_real_3_decoder_1

In [0]:
opt = Opt()
opt.num_epoches = 100
opt.useCuda = True
opt.learning_rate = 3e-6
opt.decay = 1e-5
opt.batch_size = 1
opt.model = YoloFlowLSTM(opt.batch_size)
opt.decoder = ConvLSTM((opt.encoding_size, opt.encoding_size), 5 * 4, 1024, opt.batch_size)
opt.pre_trained_yolo_path = 'log/flow_1/snapshot_encoder.tar'
# load model
check_point_encoder = torch.load('log/flow_real_3_decoder_3/snapshot_encoder_fine.tar', map_location=opt.device())
check_point_decoder = torch.load('log/flow_real_3_decoder_3/snapshot_decoder.tar', map_location=opt.device())

opt.model.load_state_dict(check_point_encoder['model_state_dict'], strict=True)
opt.decoder.load_state_dict(check_point_decoder['model_state_dict'], strict=True)
# all parameters are on the device the hidden states are not parameters
opt.model.to(opt.device())
opt.decoder.to(opt.device())
opt.yolo_loss = YoloLoss(opt.model.anchors, filter_fkt=filter_gt)
loss_params = {"grid_shape": (opt.encoding_size, opt.encoding_size),
               "image_shape": (opt.image_size, opt.image_size),
               "path_anchors": "dataset_utils/anchors/anchors5.txt"}
opt.pred_loss = NaiveLoss(loss_params)

opt.optimizer_encoder = torch.optim.Adam(opt.model.parameters(), lr=opt.learning_rate, betas=(opt.momentum, 0.999),
                                         weight_decay=opt.decay)
opt.optimizer_decoder = torch.optim.Adam(opt.decoder.parameters(), lr=opt.learning_rate, betas=(opt.momentum, 0.999),
                                         weight_decay=opt.decay)

opt.optimizer_encoder.load_state_dict(check_point_encoder['optimizer_state_dict'])
opt.optimizer_decoder.load_state_dict(check_point_decoder['optimizer_state_dict'])
                                       

In [0]:
import random

def filter_gt_batch(gt):

    # used to filter only existing bb for loss
    t = gt[gt[:, :, 0] != 0.0]
   
    return t[:, 1:]

def train_full_3(opt):
    # setup train and eval set
    #torch.cuda.manual_seed(123)
    #torch.manual_seed(123)
    
    # log stuff
    if os.path.isdir(opt.log_path):
        shutil.rmtree(opt.log_path)
    os.makedirs(opt.log_path)
    writer = SummaryWriter(opt.log_path)

    dataset = MotBBImageSequence('dataset_utils/Mot17_test_single.txt', use_only_first_video=False)
    train_data = Subset(dataset, range(0, dataset.valid_begin))
    valid_data = Subset(dataset, range(dataset.valid_begin, len(dataset)))
    train_loader = DataLoader(train_data, batch_size=opt.batch_size, shuffle=True, num_workers=opt.num_workers, drop_last=True)
    valid_loader = DataLoader(valid_data, batch_size=opt.batch_size, shuffle=True, num_workers=opt.num_workers, drop_last=True)
    
    obs_length = 10
    pred_length = 9
    
    epoch_len = len(train_loader)

    opt.model.train()
    opt.decoder.train()
    
    for epoch in range(opt.num_epoches):
        for img_num, (gt, b, img1, img2) in enumerate(train_loader):
            opt.model.lstm_part.reinit_lstm(opt.batch_size)
            opt.decoder.reset_hidden(opt.batch_size)
            
            seq_loss = 0
            seq_ap = 0
            seq_loss_coord, seq_loss_conf, seq_loss_pred = (0.0, 0.0, 0.0)
            pred_sequence = []
            for i in range(obs_length):  
                single_gt = b[:, i].to(opt.device())
                single_img2 = img2[:, i].to(opt.device())
                single_img2_normed = preprocess(img2[:, i]).to(opt.device())
                single_img1_normed = preprocess(img1[:, i]).to(opt.device())

                double_image = torch.cat((single_img1_normed, single_img2_normed), dim=1)

                logits = opt.model((double_image, single_img2))

                loss, loss_coord, loss_conf = opt.yolo_loss(logits, single_gt)
                seq_ap += get_ap(logits, filter_gt_batch(single_gt), opt.image_size, opt.image_size, opt.model.anchors)
                seq_loss += loss
                seq_loss_conf += loss_conf.item()
                seq_loss_coord += loss_coord.item()

            prev_out = logits_to_box_params(logits.detach(), opt.model.anchors)
            # the box parameters are normalized to [0, 1]
            # at the moment [batch, anchors, ...., h* w]
            # rearrange tensor s.t [batch, h, w, anchors, ...]
            # ... == x, y, w, h, conf
            #print(f"origin mask {torch.sum(mask, dim=(0, 1))}")
            prev_out = prev_out.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                               .permute(0, 3, 4, 1, 2)
            # be carefull the position of the conf/id in the targets is 0 not 4
            # so we change it at this point to be consistent with the labels

            #mask = (prev_out[:, :, :, :, 4] > 0.1).float()
            #print(f"origin mask {torch.sum(mask, dim=(0, 3))}")
            #mask = mask.unsqueeze(-1)
            prev_out = torch.Tensor(np.roll(prev_out.cpu().numpy(), 1, axis=-1)).to(opt.device())

            opt.decoder.set_hidden(opt.model.lstm_part.hidden, opt.model.lstm_part.cell)
            
            teacher_forcing_ratio = 0.0
            use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

            for i in range(pred_length):
                # todo anchor box assignment of the target boxes does not use iou
                _, yolo_target = opt.pred_loss.to_yolo(input=gt[:, obs_length].numpy(), target=b[:, obs_length + i].numpy(), use_iou=True)
                yolo_target = torch.Tensor(yolo_target).to(opt.device())
                # target boxes are in [0, grid_h] -> normalize to 1
                # at this point i assume that the image is a square !!!
                yolo_target[:, :, :, :, 1:] = yolo_target[:, :, :, :, 1:] / opt.encoding_size
                #if i == 0:
                  #print(f"target mask {torch.sum((yolo_target[:, :, :, :, 0] > 0.0).float(), dim=(0, 3))}")
                #prev_out = prev_out * mask  # todo maybe there is a better way
                #prev_out [batch, h, w, anchors, 4] 
                input = prev_out[:, :, :, :, 1:].contiguous()\
                                                .view(opt.batch_size, opt.encoding_size, opt.encoding_size, 
                                                      len(opt.model.anchors)*4)\
                                                .permute(0, 3, 1, 2)
                pred = opt.decoder(input)
                pred = pred.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                               .permute(0, 3, 4, 1, 2)

                seq_loss_pred += opt.pred_loss.forward(pred, prev_out, yolo_target)
                pred_sequence.append((prev_out.detach().cpu().numpy(),
                                      pred.detach().cpu().numpy(),
                                      (yolo_target).detach().cpu().numpy()))
                if not use_teacher_forcing:
                    prev_out[:, :, :, :, 1:] += pred
                else:
                    prev_out[:, :, :, :, 1:] = yolo_target[:, :, :, :, 1:]

            seq_loss += seq_loss_pred
            loss = seq_loss / 16
            loss.backward()
            if img_num % 16 == 15:
                opt.optimizer_encoder.step()
                opt.optimizer_decoder.step()

                opt.optimizer_encoder.zero_grad()
                opt.optimizer_decoder.zero_grad()
            
            seq_loss = seq_loss.item() - seq_loss_pred.item()
            
            writeLossToSummary(writer, 'Train', seq_loss/obs_length, 
                                                seq_loss_coord / obs_length,
                                                seq_loss_conf / obs_length, 
                                                epoch * epoch_len + img_num)

            print(f"epoch:{epoch} it: {img_num}")
            print(f"loss_seq: {seq_loss/obs_length}, loss_coord: {seq_loss_coord / obs_length}, loss_conf: {seq_loss_conf / obs_length}, mAP: {seq_ap/obs_length}")

            seq_loss_pred = seq_loss_pred.item()
            box_list = prediction_to_box_list(pred_sequence)
            dis_error = displacement_error(box_list, center_distance)
            writer.add_scalar('Train/loss_pred', seq_loss_pred / pred_length, epoch * epoch_len + img_num)
            writer.add_scalar('Train/dis_err', dis_error, epoch * epoch_len + img_num)
            writer.add_scalar('Train/AP', seq_ap/obs_length, epoch * epoch_len + img_num)
            print(f"loss_pred: {seq_loss_pred / pred_length}, dis_error: {dis_error}")

            draw_pred_sequence(box_list, img2[0], pred_length, obs_length, name='train_img.png')
              
        ###############
        # VALIDATION
        ###############
        print("###############")
        print("# VALIDATION BEGIN")
        print("###############")
        opt.model.eval()
        opt.decoder.eval()
        valid_len = len(valid_loader)
        loss_val = 0
        loss_ap = 0
        loss_coord_val = 0
        loss_conf_val = 0
        loss_pred_val = 0
        dis_error_val = 0
        for img_num, (gt, b, img1, img2) in enumerate(valid_loader):
            opt.model.lstm_part.reinit_lstm(opt.batch_size)
            opt.decoder.reset_hidden(opt.batch_size)
            opt.optimizer_encoder.zero_grad()
            opt.optimizer_decoder.zero_grad()
            
            seq_loss = 0
            seq_loss_coord, seq_loss_conf, seq_loss_pred = (0.0, 0.0, 0.0)
            seq_ap = 0
            pred_sequence = []
            for i in range(obs_length):
                single_gt = b[:, i].to(opt.device())
                single_img2 = img2[:, i].to(opt.device())
                single_img2_normed = preprocess(img2[:, i]).to(opt.device())
                single_img1_normed = preprocess(img1[:, i]).to(opt.device())

                double_image = torch.cat((single_img1_normed, single_img2_normed), dim=1)
                with torch.no_grad():
                    logits = opt.model((double_image, single_img2))

                    loss, loss_coord, loss_conf = opt.yolo_loss(logits, single_gt)
                    
                seq_ap += get_ap(logits.detach(), filter_gt_batch(single_gt), opt.image_size, opt.image_size, opt.model.anchors)
                seq_loss += loss
                seq_loss_conf += loss_conf.item()
                seq_loss_coord += loss_coord.item()

            with torch.no_grad():
                prev_out = logits_to_box_params(logits.detach(), opt.model.anchors)
                # the box parameters are normalized to [0, 1]
                # at the moment [batch, anchors, ...., h* w]
                # rearrange tensor s.t [batch, h, w, anchors, ...]
                # ... == x, y, w, h, conf
                #print(f"origin mask {torch.sum(mask, dim=(0, 1))}")
                prev_out = prev_out.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                                   .permute(0, 3, 4, 1, 2)
                # be carefull the position of the conf/id in the targets is 0 not 4
                # so we change it at this point to be consistent with the labels

                #mask = (prev_out[:, :, :, :, 4] > 0.25).float()
                #print(f"origin mask {torch.sum(mask, dim=(0, 3))}")
                #mask = mask.unsqueeze(-1)
                prev_out = torch.Tensor(np.roll(prev_out.cpu().numpy(), 1, axis=-1)).to(opt.device())

                opt.decoder.set_hidden(opt.model.lstm_part.hidden, opt.model.lstm_part.cell)
                for i in range(pred_length):
                    # todo anchor box assignment of the target boxes does not use iou
                    _, yolo_target = opt.pred_loss.to_yolo(input=gt[:, obs_length].numpy(), target=b[:, obs_length + i].numpy(), use_iou=True)
                    yolo_target = torch.Tensor(yolo_target).to(opt.device())
                    # target boxes are in [0, grid_h] -> normalize to 1
                    # at this point i assume that the image is a square !!!
                    yolo_target[:, :, :, :, 1:] = yolo_target[:, :, :, :, 1:] / opt.encoding_size
                    #if i == 0:
                      #print(f"target mask {torch.sum((yolo_target[:, :, :, :, 0] > 0.0).float(), dim=(0, 3))}")
                    #prev_out = prev_out * mask  # todo maybe there is a better way
                    input = prev_out[:, :, :, :, 1:].contiguous()\
                                                    .view(opt.batch_size, opt.encoding_size, opt.encoding_size, 
                                                          len(opt.model.anchors)*4)\
                                                    .permute(0, 3, 1, 2)
                    pred = opt.decoder(input)
                    pred = pred.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                                   .permute(0, 3, 4, 1, 2)

                    seq_loss_pred += opt.pred_loss.forward(pred, prev_out, yolo_target)
                    pred_sequence.append((prev_out.detach().cpu().numpy(),
                                          pred.detach().cpu().numpy(),
                                          (yolo_target).detach().cpu().numpy()))
                    prev_out[:, :, :, :, 1:] += pred


                seq_loss_pred = seq_loss_pred.item()
                box_list = prediction_to_box_list(pred_sequence)
                dis_error = displacement_error(box_list, center_distance)
            
            loss_val += seq_loss.item() /valid_len /obs_length
            loss_coord_val += seq_loss_coord /valid_len /obs_length
            loss_conf_val += seq_loss_conf /valid_len /obs_length
            loss_ap += seq_ap / obs_length / valid_len

            loss_pred_val += seq_loss_pred /valid_len  / pred_length
            dis_error_val += dis_error /valid_len
        draw_pred_sequence(box_list, img2[0], pred_length, obs_length, name=f'val_img.png')
        writeLossToSummary(writer, 'Val', loss_val, loss_coord_val,
                           loss_conf_val, (epoch+1) * epoch_len)

        print(f"epoch:{epoch}")
        print(f"loss_seq: {loss_val}, loss_coord: {loss_coord_val}, loss_conf: {loss_conf_val}, mAP: {loss_ap}")

        box_list = prediction_to_box_list(pred_sequence)
        dis_error = displacement_error(box_list, center_distance)
        writer.add_scalar('Val/loss_pred', loss_pred_val, (epoch+1) * epoch_len)
        writer.add_scalar('Val/dis_err', dis_error_val, (epoch+1) * epoch_len)
        writer.add_scalar('Val/mAP', loss_ap, (epoch+1) * epoch_len)

        print(f"loss_pred: {loss_pred_val}, dis_error: {dis_error_val}")
        print("###############")
        print("# VALIDATION END")
        print("###############")
        
        torch.save({'epoch': epoch, 'model_state_dict': opt.model.state_dict(),
                    'optimizer_state_dict': opt.optimizer_encoder.state_dict()},
                   opt.log_path+'/snapshot_encoder_fine.tar')
        torch.save({'epoch': epoch, 'model_state_dict': opt.decoder.state_dict(),
                    'optimizer_state_dict': opt.optimizer_decoder.state_dict()},
                   opt.log_path+'/snapshot_decoder.tar')

    writer.close()

In [0]:
import warnings
warnings.filterwarnings('ignore')

train_full_3(opt)

finished video 0 at index 470/470
finished video 1 at index 830/830
finished video 2 at index 660/659
finished video 3 at index 410/410
finished video 4 at index 515/513
finished video 5 at index 710/710
finished video 6 at index 590/590
finished video 0 at index 120/120
finished video 1 at index 210/210
finished video 2 at index 170/168
finished video 3 at index 105/105
finished video 4 at index 135/131
finished video 5 at index 180/180
finished video 6 at index 150/150
Mean trajectory length: 220.06725663716813
Deviation of trajectory length: 267.3380063437236
Max trajectory length: 1050.0
Min trajectory length: 1.0
Mean displacement: 96.97458052233117
Deviation of displacements: 103.94647223357384
Max displacement: 417.3182516766331
Min displacement: 0.0
epoch:0 it: 0
loss_seq: 0.4921749121509492, loss_coord: 0.22175303101539612, loss_conf: 0.2704218924045563, mAP: 0.10416666666666667
loss_pred: 0.00131718245231443, dis_error: 13.850337028503418
epoch:0 it: 1
loss_seq: 0.66081615034

fine tune decoder 2 with augmented labels

In [0]:
opt = Opt()
opt.num_epoches = 100
opt.useCuda = True
opt.learning_rate = 3e-5
opt.decay = 1e-5
opt.batch_size = 1
opt.model = YoloFlowLSTM(opt.batch_size)
opt.decoder = ConvLSTM((opt.encoding_size, opt.encoding_size), 5 * 4, 1024, opt.batch_size)
opt.pre_trained_yolo_path = 'log/flow_1/snapshot_encoder.tar'
# load model
check_point_encoder = torch.load('log/flow_real_3_decoder_3/snapshot_encoder_fine.tar', map_location=opt.device())
check_point_decoder = torch.load('log/flow_real_3_decoder_3/snapshot_decoder.tar', map_location=opt.device())
model_state_dict_yolo = check_point_encoder['model_state_dict']
opt.model.load_state_dict(model_state_dict_yolo, strict=True)
opt.decoder.load_state_dict(check_point_decoder['model_state_dict'])
# all parameters are on the device the hidden states are not parameters
opt.model.to(opt.device())
opt.decoder.to(opt.device())
opt.yolo_loss = YoloLoss(opt.model.anchors, filter_fkt=filter_gt)
loss_params = {"grid_shape": (opt.encoding_size, opt.encoding_size),
               "image_shape": (opt.image_size, opt.image_size),
               "path_anchors": "dataset_utils/anchors/anchors5.txt"}
opt.pred_loss = NaiveLoss(loss_params)

opt.optimizer_encoder = torch.optim.Adam(opt.model.parameters(), lr=opt.learning_rate, betas=(opt.momentum, 0.999),
                                         weight_decay=opt.decay)
opt.optimizer_decoder = torch.optim.Adam(opt.decoder.parameters(), lr=opt.learning_rate, betas=(opt.momentum, 0.999),
                                         weight_decay=opt.decay)
#opt.optimizer_encoder.load_state_dict(check_point_encoder['optimizer_state_dict'])
#opt.optimizer_decoder.load_state_dict(check_point_decoder['optimizer_state_dict'])

In [0]:
import random

def filter_gt_batch(gt):

    # used to filter only existing bb for loss
    t = gt[gt[:, :, 0] != 0.0]
   
    return t[:, 1:]

def train_full_3(opt):
    # setup train and eval set
    #torch.cuda.manual_seed(123)
    #torch.manual_seed(123)
    
    # log stuff
    if os.path.isdir(opt.log_path):
        shutil.rmtree(opt.log_path)
    os.makedirs(opt.log_path)
    writer = SummaryWriter(opt.log_path)

    dataset = MotBBImageSequence('dataset_utils/Mot17_test_single.txt', use_only_first_video=False)
    train_data = Subset(dataset, range(0, dataset.valid_begin))
    valid_data = Subset(dataset, range(dataset.valid_begin, len(dataset)))
    train_loader = DataLoader(train_data, batch_size=opt.batch_size, shuffle=True, num_workers=opt.num_workers, drop_last=True)
    valid_loader = DataLoader(valid_data, batch_size=opt.batch_size, shuffle=True, num_workers=opt.num_workers, drop_last=True)
    
    obs_length = 10
    pred_length = 9
    
    epoch_len = len(train_loader)

    opt.model.train()
    opt.decoder.train()
    
    for epoch in range(opt.num_epoches):
        opt.model.train()
        opt.decoder.train()
        for img_num, (gt, b, img1, img2) in enumerate(train_loader):
            opt.model.lstm_part.reinit_lstm(opt.batch_size)
            opt.decoder.reset_hidden(opt.batch_size)
            
            seq_loss = 0
            seq_ap = 0
            seq_loss_coord, seq_loss_conf, seq_loss_pred = (0.0, 0.0, 0.0)
            pred_sequence = []
            for i in range(obs_length):  
                single_gt = b[:, i].to(opt.device())
                single_img2 = img2[:, i].to(opt.device())
                single_img2_normed = preprocess(img2[:, i]).to(opt.device())
                single_img1_normed = preprocess(img1[:, i]).to(opt.device())

                double_image = torch.cat((single_img1_normed, single_img2_normed), dim=1)

                logits = opt.model((double_image, single_img2))

                loss, loss_coord, loss_conf = opt.yolo_loss(logits, single_gt)
                seq_ap += get_ap(logits, filter_gt_batch(single_gt), opt.image_size, opt.image_size, opt.model.anchors)
                seq_loss += loss
                seq_loss_conf += loss_conf.item()
                seq_loss_coord += loss_coord.item()

            prev_out = logits_to_box_params(logits.detach(), opt.model.anchors)
            # the box parameters are normalized to [0, 1]
            # at the moment [batch, anchors, ...., h* w]
            # rearrange tensor s.t [batch, h, w, anchors, ...]
            # ... == x, y, w, h, conf
            #print(f"origin mask {torch.sum(mask, dim=(0, 1))}")
            prev_out = prev_out.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                               .permute(0, 3, 4, 1, 2)
            # be carefull the position of the conf/id in the targets is 0 not 4
            # so we change it at this point to be consistent with the labels

            #mask = (prev_out[:, :, :, :, 4] > 0.1).float()
            #print(f"origin mask {torch.sum(mask, dim=(0, 3))}")
            #mask = mask.unsqueeze(-1)
            prev_out = torch.Tensor(np.roll(prev_out.cpu().numpy(), 1, axis=-1)).to(opt.device())

            opt.decoder.set_hidden(opt.model.lstm_part.hidden, opt.model.lstm_part.cell)
            
            gt_boxes, _ = opt.pred_loss.to_yolo(input=gt[:, obs_length].numpy(), target=b[:, obs_length].numpy(), use_iou=True)
            target_correction = prev_out[:, :, :, :, 1:3] - (torch.Tensor(gt_boxes).to(opt.device())[:, :, :, :, 1:3] / opt.encoding_size)
            for i in range(pred_length):
                # todo anchor box assignment of the target boxes does not use iou
                _, yolo_target = opt.pred_loss.to_yolo(input=gt[:, obs_length].numpy(), target=b[:, obs_length + i].numpy(), use_iou=True)
                yolo_target = torch.Tensor(yolo_target).to(opt.device())
                # target boxes are in [0, grid_h] -> normalize to 1
                # at this point i assume that the image is a square !!!
                yolo_target[:, :, :, :, 1:] = yolo_target[:, :, :, :, 1:] / opt.encoding_size
                yolo_target[:, :, :, :, 1:3] += target_correction[:, :, :, :, 1:3]
                
                #if i == 0:
                  #print(f"target mask {torch.sum((yolo_target[:, :, :, :, 0] > 0.0).float(), dim=(0, 3))}")
                #prev_out = prev_out * mask  # todo maybe there is a better way
                #prev_out [batch, h, w, anchors, 4] 
                input = prev_out[:, :, :, :, 1:].contiguous()\
                                                .view(opt.batch_size, opt.encoding_size, opt.encoding_size, 
                                                      len(opt.model.anchors)*4)\
                                                .permute(0, 3, 1, 2)
                pred = opt.decoder(input)
                pred = pred.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                               .permute(0, 3, 4, 1, 2)

                seq_loss_pred += opt.pred_loss.forward(pred, prev_out, yolo_target)
                yolo_target[:, :, :, :, 1:3] -= target_correction[:, :, :, :, 1:3] # revert correction for visualization
                pred_sequence.append((prev_out.detach().cpu().numpy(),
                                      pred.detach().cpu().numpy(),
                                      (yolo_target).detach().cpu().numpy()))

                prev_out[:, :, :, :, 1:] += pred


            seq_loss += seq_loss_pred
            loss = seq_loss / 16
            loss.backward()
            if img_num % 16 == 15:
                opt.optimizer_encoder.step()
                opt.optimizer_decoder.step()

                opt.optimizer_encoder.zero_grad()
                opt.optimizer_decoder.zero_grad()
            
            seq_loss = seq_loss.item() - seq_loss_pred.item()
            
            writeLossToSummary(writer, 'Train', seq_loss/obs_length, 
                                                seq_loss_coord / obs_length,
                                                seq_loss_conf / obs_length, 
                                                epoch * epoch_len + img_num)

            print(f"epoch:{epoch} it: {img_num}")
            print(f"loss_seq: {seq_loss/obs_length}, loss_coord: {seq_loss_coord / obs_length}, loss_conf: {seq_loss_conf / obs_length}, mAP: {seq_ap/obs_length}")

            seq_loss_pred = seq_loss_pred.item()
            box_list = prediction_to_box_list(pred_sequence)
            dis_error = displacement_error(box_list, center_distance)
            writer.add_scalar('Train/loss_pred', seq_loss_pred / pred_length, epoch * epoch_len + img_num)
            writer.add_scalar('Train/dis_err', dis_error, epoch * epoch_len + img_num)
            writer.add_scalar('Train/AP', seq_ap/obs_length, epoch * epoch_len + img_num)
            print(f"loss_pred: {seq_loss_pred / pred_length}, dis_error: {dis_error}")

        draw_pred_sequence(box_list, img2[0], pred_length, obs_length, name='train_img.png')
              
        ###############
        # VALIDATION
        ###############
        print("###############")
        print("# VALIDATION BEGIN")
        print("###############")
        opt.model.eval()
        opt.decoder.eval()
        valid_len = len(valid_loader)
        loss_val = 0
        loss_ap = 0
        loss_coord_val = 0
        loss_conf_val = 0
        loss_pred_val = 0
        dis_error_val = 0
        for img_num, (gt, b, img1, img2) in enumerate(valid_loader):
            opt.model.lstm_part.reinit_lstm(opt.batch_size)
            opt.decoder.reset_hidden(opt.batch_size)
            opt.optimizer_encoder.zero_grad()
            opt.optimizer_decoder.zero_grad()
            
            seq_loss = 0
            seq_loss_coord, seq_loss_conf, seq_loss_pred = (0.0, 0.0, 0.0)
            seq_ap = 0
            pred_sequence = []
            for i in range(obs_length):
                single_gt = b[:, i].to(opt.device())
                single_img2 = img2[:, i].to(opt.device())
                single_img2_normed = preprocess(img2[:, i]).to(opt.device())
                single_img1_normed = preprocess(img1[:, i]).to(opt.device())

                double_image = torch.cat((single_img1_normed, single_img2_normed), dim=1)
                with torch.no_grad():
                    logits = opt.model((double_image, single_img2))

                    loss, loss_coord, loss_conf = opt.yolo_loss(logits, single_gt)
                    
                seq_ap += get_ap(logits.detach(), filter_gt_batch(single_gt), opt.image_size, opt.image_size, opt.model.anchors)
                seq_loss += loss
                seq_loss_conf += loss_conf.item()
                seq_loss_coord += loss_coord.item()

            with torch.no_grad():
                prev_out = logits_to_box_params(logits.detach(), opt.model.anchors)
                # the box parameters are normalized to [0, 1]
                # at the moment [batch, anchors, ...., h* w]
                # rearrange tensor s.t [batch, h, w, anchors, ...]
                # ... == x, y, w, h, conf
                #print(f"origin mask {torch.sum(mask, dim=(0, 1))}")
                prev_out = prev_out.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                                   .permute(0, 3, 4, 1, 2)
                # be carefull the position of the conf/id in the targets is 0 not 4
                # so we change it at this point to be consistent with the labels

                #mask = (prev_out[:, :, :, :, 4] > 0.25).float()
                #print(f"origin mask {torch.sum(mask, dim=(0, 3))}")
                #mask = mask.unsqueeze(-1)
                prev_out = torch.Tensor(np.roll(prev_out.cpu().numpy(), 1, axis=-1)).to(opt.device())

                opt.decoder.set_hidden(opt.model.lstm_part.hidden, opt.model.lstm_part.cell)
                for i in range(pred_length):
                    # todo anchor box assignment of the target boxes does not use iou
                    _, yolo_target = opt.pred_loss.to_yolo(input=gt[:, obs_length].numpy(), target=b[:, obs_length + i].numpy(), use_iou=True)
                    yolo_target = torch.Tensor(yolo_target).to(opt.device())
                    # target boxes are in [0, grid_h] -> normalize to 1
                    # at this point i assume that the image is a square !!!
                    yolo_target[:, :, :, :, 1:] = yolo_target[:, :, :, :, 1:] / opt.encoding_size
                    #if i == 0:
                      #print(f"target mask {torch.sum((yolo_target[:, :, :, :, 0] > 0.0).float(), dim=(0, 3))}")
                    #prev_out = prev_out * mask  # todo maybe there is a better way
                    input = prev_out[:, :, :, :, 1:].contiguous()\
                                                    .view(opt.batch_size, opt.encoding_size, opt.encoding_size, 
                                                          len(opt.model.anchors)*4)\
                                                    .permute(0, 3, 1, 2)
                    pred = opt.decoder(input)
                    pred = pred.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                                   .permute(0, 3, 4, 1, 2)

                    seq_loss_pred += opt.pred_loss.forward(pred, prev_out, yolo_target)
                    pred_sequence.append((prev_out.detach().cpu().numpy(),
                                          pred.detach().cpu().numpy(),
                                          (yolo_target).detach().cpu().numpy()))
                    prev_out[:, :, :, :, 1:] += pred


                seq_loss_pred = seq_loss_pred.item()
                box_list = prediction_to_box_list(pred_sequence)
                dis_error = displacement_error(box_list, center_distance)
            
            loss_val += seq_loss.item() /valid_len /obs_length
            loss_coord_val += seq_loss_coord /valid_len /obs_length
            loss_conf_val += seq_loss_conf /valid_len /obs_length
            loss_ap += seq_ap / obs_length / valid_len

            loss_pred_val += seq_loss_pred /valid_len  / pred_length
            dis_error_val += dis_error /valid_len
        draw_pred_sequence(box_list, img2[0], pred_length, obs_length, name=f'val_img.png')
        writeLossToSummary(writer, 'Val', loss_val, loss_coord_val,
                           loss_conf_val, (epoch+1) * epoch_len)

        print(f"epoch:{epoch}")
        print(f"loss_seq: {loss_val}, loss_coord: {loss_coord_val}, loss_conf: {loss_conf_val}, mAP: {loss_ap}")

        box_list = prediction_to_box_list(pred_sequence)
        dis_error = displacement_error(box_list, center_distance)
        writer.add_scalar('Val/loss_pred', loss_pred_val, (epoch+1) * epoch_len)
        writer.add_scalar('Val/dis_err', dis_error_val, (epoch+1) * epoch_len)
        writer.add_scalar('Val/mAP', loss_ap, (epoch+1) * epoch_len)

        print(f"loss_pred: {loss_pred_val}, dis_error: {dis_error_val}")
        print("###############")
        print("# VALIDATION END")
        print("###############")
        
        torch.save({'epoch': epoch, 'model_state_dict': opt.model.state_dict(),
                    'optimizer_state_dict': opt.optimizer_encoder.state_dict()},
                   opt.log_path+'/snapshot_encoder_fine.tar')
        torch.save({'epoch': epoch, 'model_state_dict': opt.decoder.state_dict(),
                    'optimizer_state_dict': opt.optimizer_decoder.state_dict()},
                   opt.log_path+'/snapshot_decoder.tar')

    writer.close()

In [0]:
import warnings
warnings.filterwarnings('ignore')

train_full_3(opt)

finished video 0 at index 470/470
finished video 1 at index 830/830
finished video 2 at index 660/659
finished video 3 at index 410/410
finished video 4 at index 515/513
finished video 5 at index 710/710
finished video 6 at index 590/590
finished video 0 at index 120/120
finished video 1 at index 210/210
finished video 2 at index 170/168
finished video 3 at index 105/105
finished video 4 at index 135/131
finished video 5 at index 180/180
finished video 6 at index 150/150
Mean trajectory length: 220.06725663716813
Deviation of trajectory length: 267.3380063437236
Max trajectory length: 1050.0
Min trajectory length: 1.0
Mean displacement: 96.97458052233117
Deviation of displacements: 103.94647223357384
Max displacement: 417.3182516766331
Min displacement: 0.0
epoch:0 it: 0
loss_seq: 0.42195842806249856, loss_coord: 0.1337052710354328, loss_conf: 0.2882531687617302, mAP: 0.3615384615384615
loss_pred: 0.0013164089371760686, dis_error: 10.70614242553711
epoch:0 it: 1
loss_seq: 2.24959989786

KeyboardInterrupt: ignored

In [0]:
opt = Opt()
opt.num_epoches = 100
opt.useCuda = True
opt.learning_rate = 3e-5
opt.decay = 1e-5
opt.batch_size = 1
opt.model = YoloFlowLSTM(opt.batch_size)
opt.decoder = ConvLSTM((opt.encoding_size, opt.encoding_size), 5 * 4, 1024, opt.batch_size)
opt.pre_trained_yolo_path = 'log/flow_1/snapshot_encoder.tar'
# load model
check_point_encoder = torch.load('log/flow_decoder_2/snapshot_encoder_fine.tar', map_location=opt.device())
check_point_decoder = torch.load('log/flow_decoder_2/snapshot_decoder.tar', map_location=opt.device())
model_state_dict_yolo = check_point_encoder['model_state_dict']
opt.model.load_state_dict(model_state_dict_yolo, strict=True)
opt.decoder.load_state_dict(check_point_decoder['model_state_dict'])
# all parameters are on the device the hidden states are not parameters
opt.model.to(opt.device())
opt.decoder.to(opt.device())
opt.yolo_loss = YoloLoss(opt.model.anchors, filter_fkt=filter_gt)
loss_params = {"grid_shape": (opt.encoding_size, opt.encoding_size),
               "image_shape": (opt.image_size, opt.image_size),
               "path_anchors": "dataset_utils/anchors/anchors5.txt"}
opt.pred_loss = NaiveLoss(loss_params)

opt.optimizer_encoder = torch.optim.Adam(opt.model.parameters(), lr=opt.learning_rate, betas=(opt.momentum, 0.999),
                                         weight_decay=opt.decay)
opt.optimizer_decoder = torch.optim.Adam(opt.decoder.parameters(), lr=opt.learning_rate, betas=(opt.momentum, 0.999),
                                         weight_decay=opt.decay)
#opt.optimizer_encoder.load_state_dict(check_point_encoder['optimizer_state_dict'])
#opt.optimizer_decoder.load_state_dict(check_point_decoder['optimizer_state_dict'])

In [0]:
import random

def filter_gt_batch(gt):

    # used to filter only existing bb for loss
    t = gt[gt[:, :, 0] != 0.0]
   
    return t[:, 1:]

def train_full_3(opt):
    # setup train and eval set
    #torch.cuda.manual_seed(123)
    #torch.manual_seed(123)
    
    # log stuff
    if os.path.isdir(opt.log_path):
        shutil.rmtree(opt.log_path)
    os.makedirs(opt.log_path)
    writer = SummaryWriter(opt.log_path)

    dataset = MotBBImageSequence('dataset_utils/Mot17_test_single.txt', use_only_first_video=False)
    train_data = Subset(dataset, range(0, dataset.valid_begin))
    valid_data = Subset(dataset, range(dataset.valid_begin, len(dataset)))
    train_loader = DataLoader(train_data, batch_size=opt.batch_size, shuffle=True, num_workers=opt.num_workers, drop_last=True)
    valid_loader = DataLoader(valid_data, batch_size=opt.batch_size, shuffle=True, num_workers=opt.num_workers, drop_last=True)
    
    obs_length = 10
    pred_length = 9
    
    epoch_len = len(train_loader)
    
    for epoch in range(opt.num_epoches):
        opt.model.train()
        opt.decoder.train()
        for img_num, (gt, b, img1, img2) in enumerate(train_loader):
            opt.model.lstm_part.reinit_lstm(opt.batch_size)
            opt.decoder.reset_hidden(opt.batch_size)
            
            seq_loss = 0
            seq_ap = 0
            seq_loss_coord, seq_loss_conf, seq_loss_pred = (0.0, 0.0, 0.0)
            pred_sequence = []
            for i in range(obs_length):  
                single_gt = b[:, i].to(opt.device())
                single_img2 = img2[:, i].to(opt.device())
                single_img2_normed = preprocess(img2[:, i]).to(opt.device())
                single_img1_normed = preprocess(img1[:, i]).to(opt.device())

                double_image = torch.cat((single_img1_normed, single_img2_normed), dim=1)

                logits = opt.model((double_image, single_img2))

                loss, loss_coord, loss_conf = opt.yolo_loss(logits, single_gt)
                seq_ap += get_ap(logits, filter_gt_batch(single_gt), opt.image_size, opt.image_size, opt.model.anchors)
                seq_loss += loss
                seq_loss_conf += loss_conf.item()
                seq_loss_coord += loss_coord.item()

            prev_out = logits_to_box_params(logits.detach(), opt.model.anchors)
            # the box parameters are normalized to [0, 1]
            # at the moment [batch, anchors, ...., h* w]
            # rearrange tensor s.t [batch, h, w, anchors, ...]
            # ... == x, y, w, h, conf
            #print(f"origin mask {torch.sum(mask, dim=(0, 1))}")
            prev_out = prev_out.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                               .permute(0, 3, 4, 1, 2)
            # be carefull the position of the conf/id in the targets is 0 not 4
            # so we change it at this point to be consistent with the labels

            #mask = (prev_out[:, :, :, :, 4] > 0.1).float()
            #print(f"origin mask {torch.sum(mask, dim=(0, 3))}")
            #mask = mask.unsqueeze(-1)
            prev_out = torch.Tensor(np.roll(prev_out.cpu().numpy(), 1, axis=-1)).to(opt.device())

            opt.decoder.set_hidden(opt.model.lstm_part.hidden, opt.model.lstm_part.cell)
            
            gt_boxes, _ = opt.pred_loss.to_yolo(input=gt[:, obs_length].numpy(), target=b[:, obs_length].numpy(), use_iou=True)
            target_correction = prev_out[:, :, :, :, 1:3] - (torch.Tensor(gt_boxes).to(opt.device())[:, :, :, :, 1:3] / opt.encoding_size)
            for i in range(pred_length):
                # todo anchor box assignment of the target boxes does not use iou
                _, yolo_target = opt.pred_loss.to_yolo(input=gt[:, obs_length].numpy(), target=b[:, obs_length + i].numpy(), use_iou=True)
                yolo_target = torch.Tensor(yolo_target).to(opt.device())
                # target boxes are in [0, grid_h] -> normalize to 1
                # at this point i assume that the image is a square !!!
                yolo_target[:, :, :, :, 1:] = yolo_target[:, :, :, :, 1:] / opt.encoding_size
                yolo_target[:, :, :, :, 1:3] += target_correction[:, :, :, :, 1:3]
                
                #if i == 0:
                  #print(f"target mask {torch.sum((yolo_target[:, :, :, :, 0] > 0.0).float(), dim=(0, 3))}")
                #prev_out = prev_out * mask  # todo maybe there is a better way
                #prev_out [batch, h, w, anchors, 4] 
                input = prev_out[:, :, :, :, 1:].contiguous()\
                                                .view(opt.batch_size, opt.encoding_size, opt.encoding_size, 
                                                      len(opt.model.anchors)*4)\
                                                .permute(0, 3, 1, 2)
                pred = opt.decoder(input)
                pred = pred.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                               .permute(0, 3, 4, 1, 2)

                seq_loss_pred += opt.pred_loss.forward(pred, prev_out, yolo_target)
                yolo_target[:, :, :, :, 1:3] -= target_correction[:, :, :, :, 1:3] # revert correction for visualization
                pred_sequence.append((prev_out.detach().cpu().numpy(),
                                      pred.detach().cpu().numpy(),
                                      (yolo_target).detach().cpu().numpy()))

                prev_out[:, :, :, :, 1:] += pred


            seq_loss += seq_loss_pred
            loss = seq_loss / 16
            loss.backward()
            if img_num % 16 == 15:
                opt.optimizer_encoder.step()
                opt.optimizer_decoder.step()

                opt.optimizer_encoder.zero_grad()
                opt.optimizer_decoder.zero_grad()
            
            seq_loss = seq_loss.item() - seq_loss_pred.item()
            
            writeLossToSummary(writer, 'Train', seq_loss/obs_length, 
                                                seq_loss_coord / obs_length,
                                                seq_loss_conf / obs_length, 
                                                epoch * epoch_len + img_num)

            print(f"epoch:{epoch} it: {img_num}")
            print(f"loss_seq: {seq_loss/obs_length}, loss_coord: {seq_loss_coord / obs_length}, loss_conf: {seq_loss_conf / obs_length}, mAP: {seq_ap/obs_length}")

            seq_loss_pred = seq_loss_pred.item()
            box_list = prediction_to_box_list(pred_sequence)
            dis_error = displacement_error(box_list, center_distance)
            writer.add_scalar('Train/loss_pred', seq_loss_pred / pred_length, epoch * epoch_len + img_num)
            writer.add_scalar('Train/dis_err', dis_error, epoch * epoch_len + img_num)
            writer.add_scalar('Train/AP', seq_ap/obs_length, epoch * epoch_len + img_num)
            print(f"loss_pred: {seq_loss_pred / pred_length}, dis_error: {dis_error}")

        draw_pred_sequence(box_list, img2[0], pred_length, obs_length, name='train_img.png')
              
        ###############
        # VALIDATION
        ###############
        print("###############")
        print("# VALIDATION BEGIN")
        print("###############")
        opt.model.eval()
        opt.decoder.eval()
        valid_len = len(valid_loader)
        loss_val = 0
        loss_ap = 0
        loss_coord_val = 0
        loss_conf_val = 0
        loss_pred_val = 0
        dis_error_val = 0
        for img_num, (gt, b, img1, img2) in enumerate(valid_loader):
            opt.model.lstm_part.reinit_lstm(opt.batch_size)
            opt.decoder.reset_hidden(opt.batch_size)
            opt.optimizer_encoder.zero_grad()
            opt.optimizer_decoder.zero_grad()
            
            seq_loss = 0
            seq_loss_coord, seq_loss_conf, seq_loss_pred = (0.0, 0.0, 0.0)
            seq_ap = 0
            pred_sequence = []
            for i in range(obs_length):
                single_gt = b[:, i].to(opt.device())
                single_img2 = img2[:, i].to(opt.device())
                single_img2_normed = preprocess(img2[:, i]).to(opt.device())
                single_img1_normed = preprocess(img1[:, i]).to(opt.device())

                double_image = torch.cat((single_img1_normed, single_img2_normed), dim=1)
                with torch.no_grad():
                    logits = opt.model((double_image, single_img2))

                    loss, loss_coord, loss_conf = opt.yolo_loss(logits, single_gt)
                    
                seq_ap += get_ap(logits.detach(), filter_gt_batch(single_gt), opt.image_size, opt.image_size, opt.model.anchors)
                seq_loss += loss
                seq_loss_conf += loss_conf.item()
                seq_loss_coord += loss_coord.item()

            with torch.no_grad():
                prev_out = logits_to_box_params(logits.detach(), opt.model.anchors)
                # the box parameters are normalized to [0, 1]
                # at the moment [batch, anchors, ...., h* w]
                # rearrange tensor s.t [batch, h, w, anchors, ...]
                # ... == x, y, w, h, conf
                #print(f"origin mask {torch.sum(mask, dim=(0, 1))}")
                prev_out = prev_out.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                                   .permute(0, 3, 4, 1, 2)
                # be carefull the position of the conf/id in the targets is 0 not 4
                # so we change it at this point to be consistent with the labels

                #mask = (prev_out[:, :, :, :, 4] > 0.25).float()
                #print(f"origin mask {torch.sum(mask, dim=(0, 3))}")
                #mask = mask.unsqueeze(-1)
                prev_out = torch.Tensor(np.roll(prev_out.cpu().numpy(), 1, axis=-1)).to(opt.device())

                opt.decoder.set_hidden(opt.model.lstm_part.hidden, opt.model.lstm_part.cell)
                for i in range(pred_length):
                    # todo anchor box assignment of the target boxes does not use iou
                    _, yolo_target = opt.pred_loss.to_yolo(input=gt[:, obs_length].numpy(), target=b[:, obs_length + i].numpy(), use_iou=True)
                    yolo_target = torch.Tensor(yolo_target).to(opt.device())
                    # target boxes are in [0, grid_h] -> normalize to 1
                    # at this point i assume that the image is a square !!!
                    yolo_target[:, :, :, :, 1:] = yolo_target[:, :, :, :, 1:] / opt.encoding_size
                    #if i == 0:
                      #print(f"target mask {torch.sum((yolo_target[:, :, :, :, 0] > 0.0).float(), dim=(0, 3))}")
                    #prev_out = prev_out * mask  # todo maybe there is a better way
                    input = prev_out[:, :, :, :, 1:].contiguous()\
                                                    .view(opt.batch_size, opt.encoding_size, opt.encoding_size, 
                                                          len(opt.model.anchors)*4)\
                                                    .permute(0, 3, 1, 2)
                    pred = opt.decoder(input)
                    pred = pred.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                                   .permute(0, 3, 4, 1, 2)

                    seq_loss_pred += opt.pred_loss.forward(pred, prev_out, yolo_target)
                    pred_sequence.append((prev_out.detach().cpu().numpy(),
                                          pred.detach().cpu().numpy(),
                                          (yolo_target).detach().cpu().numpy()))
                    prev_out[:, :, :, :, 1:] += pred


                seq_loss_pred = seq_loss_pred.item()
                box_list = prediction_to_box_list(pred_sequence)
                dis_error = displacement_error(box_list, center_distance)
            
            loss_val += seq_loss.item() /valid_len /obs_length
            loss_coord_val += seq_loss_coord /valid_len /obs_length
            loss_conf_val += seq_loss_conf /valid_len /obs_length
            loss_ap += seq_ap / obs_length / valid_len

            loss_pred_val += seq_loss_pred /valid_len  / pred_length
            dis_error_val += dis_error /valid_len
        draw_pred_sequence(box_list, img2[0], pred_length, obs_length, name=f'val_img.png')
        writeLossToSummary(writer, 'Val', loss_val, loss_coord_val,
                           loss_conf_val, (epoch+1) * epoch_len)

        print(f"epoch:{epoch}")
        print(f"loss_seq: {loss_val}, loss_coord: {loss_coord_val}, loss_conf: {loss_conf_val}, mAP: {loss_ap}")

        box_list = prediction_to_box_list(pred_sequence)
        dis_error = displacement_error(box_list, center_distance)
        writer.add_scalar('Val/loss_pred', loss_pred_val, (epoch+1) * epoch_len)
        writer.add_scalar('Val/dis_err', dis_error_val, (epoch+1) * epoch_len)
        writer.add_scalar('Val/mAP', loss_ap, (epoch+1) * epoch_len)

        print(f"loss_pred: {loss_pred_val}, dis_error: {dis_error_val}")
        print("###############")
        print("# VALIDATION END")
        print("###############")
        
        torch.save({'epoch': epoch, 'model_state_dict': opt.model.state_dict(),
                    'optimizer_state_dict': opt.optimizer_encoder.state_dict()},
                   opt.log_path+'/snapshot_encoder_fine.tar')
        torch.save({'epoch': epoch, 'model_state_dict': opt.decoder.state_dict(),
                    'optimizer_state_dict': opt.optimizer_decoder.state_dict()},
                   opt.log_path+'/snapshot_decoder.tar')

    writer.close()

In [0]:
def mean_squared_trajectory_error(pred_box_list, metric, image_size=416.0):
    # shape (batch_size, 120, boxes) boxes: (5) id, x, y, w, h

    error = []
    for frame in pred_box_list:
        _, output, target = frame
        for batch_id in range(len(target)):
            for box, box_id in enumerate(target[batch_id, :, 0]):
                if box_id != 0:
                    target_box = np.where(output[batch_id, :, 0] == box_id)
                    target_box = output[batch_id, target_box[0][0], 1:] * image_size
                    error.append(metric(target_box, target[batch_id, box, 1:]*image_size))
    return np.mean(error)

In [0]:
import warnings
warnings.filterwarnings('ignore')

train_full_3(opt)

finished video 0 at index 470/470
finished video 1 at index 830/830
finished video 2 at index 660/659
finished video 3 at index 410/410
finished video 4 at index 515/513
finished video 5 at index 710/710
finished video 6 at index 590/590
finished video 0 at index 120/120
finished video 1 at index 210/210
finished video 2 at index 170/168
finished video 3 at index 105/105
finished video 4 at index 135/131
finished video 5 at index 180/180
finished video 6 at index 150/150
Mean trajectory length: 220.06725663716813
Deviation of trajectory length: 267.3380063437236
Max trajectory length: 1050.0
Min trajectory length: 1.0
Mean displacement: 96.97458052233117
Deviation of displacements: 103.94647223357384
Max displacement: 417.3182516766331
Min displacement: 0.0
epoch:0 it: 0
loss_seq: 0.9512565858662129, loss_coord: 0.22395934611558915, loss_conf: 0.7272971957921982, mAP: 0.4170814246165059
loss_pred: 0.0023926355772548253, dis_error: 17.492881774902344
epoch:0 it: 1
loss_seq: 1.1143341042

test flow_real_decoder

In [21]:
def filter_gt_batch(gt):

    # used to filter only existing bb for loss
    t = gt[gt[:, :, 0] != 0.0]

    return t[:, 1:]

def test_flow_real_decoder(opt):

    dataset = MotBBImageSequence('dataset_utils/Mot17_test_single.txt', use_only_first_video=False)
    valid_data = Subset(dataset, range(dataset.valid_begin, len(dataset)))
    valid_loader = DataLoader(valid_data, batch_size=opt.batch_size, shuffle=False, num_workers=opt.num_workers, drop_last=True)

    obs_length = 10
    pred_length = 9


    print("###############")
    print("# VALIDATION BEGIN")
    print("###############")
    opt.model.eval()
    opt.decoder.eval()
    valid_len = len(valid_loader)
    loss_val = 0
    loss_ap = 0
    loss_coord_val = 0
    loss_conf_val = 0
    loss_pred_val = 0
    dis_error_val = 0
    traj_error_val = 0
    iou_val = 0
    dis_error_val_2 = 0
    traj_error_val_2 = 0
    iou_val_2 = 0
    for img_num, (gt, b, img1, img2) in enumerate(valid_loader):
        opt.model.lstm_part.reinit_lstm(opt.batch_size)
        opt.decoder.reset_hidden(opt.batch_size)

        seq_loss = 0
        seq_loss_coord, seq_loss_conf, seq_loss_pred = (0.0, 0.0, 0.0)
        seq_ap = 0
        pred_sequence = []
        for i in range(obs_length):
            single_gt = b[:, i].to(opt.device())
            single_img2 = img2[:, i].to(opt.device())
            single_img2_normed = preprocess(img2[:, i]).to(opt.device())
            single_img1_normed = preprocess(img1[:, i]).to(opt.device())

            double_image = torch.cat((single_img1_normed, single_img2_normed), dim=1)
            with torch.no_grad():
                logits = opt.model((double_image, single_img2))

                loss, loss_coord, loss_conf = opt.yolo_loss(logits, single_gt)

            seq_ap += get_ap(logits.detach(), filter_gt_batch(single_gt), opt.image_size, opt.image_size,
                             opt.model.anchors)
            seq_loss += loss
            seq_loss_conf += loss_conf.item()
            seq_loss_coord += loss_coord.item()

        with torch.no_grad():
            prev_out = logits_to_box_params(logits.detach(), opt.model.anchors)
            # the box parameters are normalized to [0, 1]
            # at the moment [batch, anchors, ...., h* w]
            # rearrange tensor s.t [batch, h, w, anchors, ...]
            # ... == x, y, w, h, conf
            # print(f"origin mask {torch.sum(mask, dim=(0, 1))}")
            prev_out = prev_out.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size,
                                     opt.encoding_size) \
                .permute(0, 3, 4, 1, 2)
            # be carefull the position of the conf/id in the targets is 0 not 4
            # so we change it at this point to be consistent with the labels

            # mask = (prev_out[:, :, :, :, 4] > 0.25).float()
            # print(f"origin mask {torch.sum(mask, dim=(0, 3))}")
            # mask = mask.unsqueeze(-1)
            prev_out = torch.Tensor(np.roll(prev_out.cpu().numpy(), 1, axis=-1)).to(opt.device())

            opt.decoder.set_hidden(opt.model.lstm_part.hidden, opt.model.lstm_part.cell)
            for i in range(pred_length):
                # todo anchor box assignment of the target boxes does not use iou
                _, yolo_target = opt.pred_loss.to_yolo(input=gt[:, obs_length].numpy(),
                                                       target=b[:, obs_length + i].numpy(), use_iou=True)
                yolo_target = torch.Tensor(yolo_target).to(opt.device())
                # target boxes are in [0, grid_h] -> normalize to 1
                # at this point i assume that the image is a square !!!
                yolo_target[:, :, :, :, 1:] = yolo_target[:, :, :, :, 1:] / opt.encoding_size
                # if i == 0:
                # print(f"target mask {torch.sum((yolo_target[:, :, :, :, 0] > 0.0).float(), dim=(0, 3))}")
                # prev_out = prev_out * mask  # todo maybe there is a better way
                input = prev_out[:, :, :, :, 1:].contiguous() \
                    .view(opt.batch_size, opt.encoding_size, opt.encoding_size,
                          len(opt.model.anchors) * 4) \
                    .permute(0, 3, 1, 2)
                pred = opt.decoder(input)
                pred = pred.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size) \
                           .permute(0, 3, 4, 1, 2)

                seq_loss_pred += opt.pred_loss.forward(pred, prev_out, yolo_target)
                pred_sequence.append((prev_out.detach().cpu().numpy(),
                                      pred.detach().cpu().numpy(),
                                      (yolo_target).detach().cpu().numpy()))
                prev_out[:, :, :, :, 1:] += pred

            seq_loss_pred = seq_loss_pred.item()
            box_list   = prediction_to_box_list(pred_sequence)
            dis_error  = displacement_error(box_list, center_distance)
            traj_error = mean_squared_trajectory_error(box_list, center_distance)
            iou        = mean_iou(box_list, image_size=416)
            dis_error_2 = displacement_error(box_list, center_distance, image_size=832)
            traj_error_2 = mean_squared_trajectory_error(box_list, center_distance, image_size=832)
            iou_2        = mean_iou(box_list, image_size=832)

        loss_val += seq_loss.item() / valid_len / obs_length
        loss_coord_val += seq_loss_coord / valid_len / obs_length
        loss_conf_val += seq_loss_conf / valid_len / obs_length
        loss_ap += seq_ap / obs_length / valid_len

        loss_pred_val  += seq_loss_pred / valid_len / pred_length
        dis_error_val  += dis_error     / valid_len
        traj_error_val += traj_error    / valid_len
        iou_val        += iou           / valid_len
        dis_error_val_2  += dis_error_2     / valid_len
        traj_error_val_2 += traj_error_2    / valid_len
        iou_val_2        += iou_2           / valid_len
        #draw_pred_sequence_2(box_list, img2[0], pred_length, obs_length, name=f'log/flow_real_decoder/imgs/val_img_{img_num}.png')

    print(f"loss_seq: {loss_val}, loss_coord: {loss_coord_val}, loss_conf: {loss_conf_val}, mAP: {loss_ap}")

    print(f"loss_pred: {loss_pred_val}, dis_error: {dis_error_val}, traj_error: {traj_error_val}, iou: {iou_val}")
    print(f"                            dis_error: {dis_error_val_2}, traj_error: {traj_error_val_2}, iou: {iou_val_2}")




opt = Opt()
opt.useCuda = True
opt.batch_size = 1
opt.num_workers = 4
opt.model = YoloFlowLSTM(opt.batch_size)
opt.decoder = ConvLSTM((opt.encoding_size, opt.encoding_size), 5 * 4, 1024, opt.batch_size)
opt.pre_trained_yolo_path = 'log/flow_1/snapshot_encoder.tar'
# load model
check_point_encoder = torch.load('log/flow_real_decoder/snapshot_encoder_fine.tar', map_location=opt.device())
check_point_decoder = torch.load('log/flow_real_decoder/snapshot_decoder.tar', map_location=opt.device())
model_state_dict_yolo = check_point_encoder['model_state_dict']
opt.model.load_state_dict(model_state_dict_yolo, strict=True)
opt.decoder.load_state_dict(check_point_decoder['model_state_dict'])
# all parameters are on the device the hidden states are not parameters
opt.model.to(opt.device())
opt.decoder.to(opt.device())
opt.yolo_loss = YoloLoss(opt.model.anchors, filter_fkt=filter_gt)
loss_params = {"grid_shape": (opt.encoding_size, opt.encoding_size),
               "image_shape": (opt.image_size, opt.image_size),
               "path_anchors": "dataset_utils/anchors/anchors5.txt"}
opt.pred_loss = NaiveLoss(loss_params)

test_flow_real_decoder(opt)

finished video 0 at index 470/470
finished video 1 at index 830/830
finished video 2 at index 660/659
finished video 3 at index 410/410
finished video 4 at index 515/513
finished video 5 at index 710/710
finished video 6 at index 590/590
finished video 0 at index 120/120
finished video 1 at index 210/210
finished video 2 at index 170/168
finished video 3 at index 105/105
finished video 4 at index 135/131
finished video 5 at index 180/180
finished video 6 at index 150/150
Mean trajectory length: 220.06725663716813
Deviation of trajectory length: 267.3380063437236
Max trajectory length: 1050.0
Min trajectory length: 1.0
Mean displacement: 96.97458052233117
Deviation of displacements: 103.94647223357384
Max displacement: 417.3182516766331
Min displacement: 0.0
###############
# VALIDATION BEGIN
###############


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


loss_seq: 4.878529923962007, loss_coord: 1.4261903645570884, loss_conf: 3.4523395666391967, mAP: 0.27793945176626395
loss_pred: 0.0049033267278860695, dis_error: 15.62708355790825, traj_error: 12.103465667334937, iou: 0.4074669688566877
                            dis_error: 31.2541671158165, traj_error: 24.206931334669875, iou: 0.40044302374049984


Test flow_real_3_decoder

In [27]:
def filter_gt_batch(gt):

    # used to filter only existing bb for loss
    t = gt[gt[:, :, 0] != 0.0]

    return t[:, 1:]

def test_flow_real_decoder(opt):

    dataset = MotBBImageSequence('dataset_utils/Mot17_test_single.txt', use_only_first_video=False)
    valid_data = Subset(dataset, range(dataset.valid_begin, len(dataset)))
    valid_loader = DataLoader(valid_data, batch_size=opt.batch_size, shuffle=False, num_workers=opt.num_workers, drop_last=True)

    obs_length = 10
    pred_length = 9


    print("###############")
    print("# VALIDATION BEGIN")
    print("###############")
    opt.model.eval()
    opt.decoder.eval()
    valid_len = len(valid_loader)
    loss_val = 0
    loss_ap = 0
    loss_coord_val = 0
    loss_conf_val = 0
    loss_pred_val = 0
    dis_error_val = 0
    traj_error_val = 0
    iou_val = 0
    dis_error_val_2 = 0
    traj_error_val_2 = 0
    iou_val_2 = 0
    for img_num, (gt, b, img1, img2) in enumerate(valid_loader):
        opt.model.lstm_part.reinit_lstm(opt.batch_size)
        opt.decoder.reset_hidden(opt.batch_size)

        seq_loss = 0
        seq_loss_coord, seq_loss_conf, seq_loss_pred = (0.0, 0.0, 0.0)
        seq_ap = 0
        pred_sequence = []
        for i in range(obs_length):
            single_gt = b[:, i].to(opt.device())
            single_img2 = img2[:, i].to(opt.device())
            single_img2_normed = preprocess(img2[:, i]).to(opt.device())
            single_img1_normed = preprocess(img1[:, i]).to(opt.device())

            double_image = torch.cat((single_img1_normed, single_img2_normed), dim=1)
            with torch.no_grad():
                logits = opt.model((double_image, single_img2))

                loss, loss_coord, loss_conf = opt.yolo_loss(logits, single_gt)

            seq_ap += get_ap(logits.detach(), filter_gt_batch(single_gt), opt.image_size, opt.image_size,
                             opt.model.anchors)
            seq_loss += loss
            seq_loss_conf += loss_conf.item()
            seq_loss_coord += loss_coord.item()

        with torch.no_grad():
            prev_out = logits_to_box_params(logits.detach(), opt.model.anchors)
            # the box parameters are normalized to [0, 1]
            # at the moment [batch, anchors, ...., h* w]
            # rearrange tensor s.t [batch, h, w, anchors, ...]
            # ... == x, y, w, h, conf
            # print(f"origin mask {torch.sum(mask, dim=(0, 1))}")
            prev_out = prev_out.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size,
                                     opt.encoding_size) \
                .permute(0, 3, 4, 1, 2)
            # be carefull the position of the conf/id in the targets is 0 not 4
            # so we change it at this point to be consistent with the labels

            # mask = (prev_out[:, :, :, :, 4] > 0.25).float()
            # print(f"origin mask {torch.sum(mask, dim=(0, 3))}")
            # mask = mask.unsqueeze(-1)
            prev_out = torch.Tensor(np.roll(prev_out.cpu().numpy(), 1, axis=-1)).to(opt.device())

            opt.decoder.set_hidden(opt.model.lstm_part.hidden, opt.model.lstm_part.cell)
            for i in range(pred_length):
                # todo anchor box assignment of the target boxes does not use iou
                _, yolo_target = opt.pred_loss.to_yolo(input=gt[:, obs_length].numpy(),
                                                       target=b[:, obs_length + i].numpy(), use_iou=True)
                yolo_target = torch.Tensor(yolo_target).to(opt.device())
                # target boxes are in [0, grid_h] -> normalize to 1
                # at this point i assume that the image is a square !!!
                yolo_target[:, :, :, :, 1:] = yolo_target[:, :, :, :, 1:] / opt.encoding_size
                # if i == 0:
                # print(f"target mask {torch.sum((yolo_target[:, :, :, :, 0] > 0.0).float(), dim=(0, 3))}")
                # prev_out = prev_out * mask  # todo maybe there is a better way
                input = prev_out[:, :, :, :, 1:].contiguous() \
                    .view(opt.batch_size, opt.encoding_size, opt.encoding_size,
                          len(opt.model.anchors) * 4) \
                    .permute(0, 3, 1, 2)
                pred = opt.decoder(input)
                pred = pred.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size) \
                           .permute(0, 3, 4, 1, 2)

                seq_loss_pred += opt.pred_loss.forward(pred, prev_out, yolo_target)
                pred_sequence.append((prev_out.detach().cpu().numpy(),
                                      pred.detach().cpu().numpy(),
                                      (yolo_target).detach().cpu().numpy()))
                prev_out[:, :, :, :, 1:] += pred

            seq_loss_pred = seq_loss_pred.item()
            box_list   = prediction_to_box_list(pred_sequence)
            dis_error  = displacement_error(box_list, center_distance)
            traj_error = mean_squared_trajectory_error(box_list, center_distance)
            iou        = mean_iou(box_list, image_size=416)
            
            dis_error_2 = displacement_error(box_list, center_distance, image_size=832)
            traj_error_2 = mean_squared_trajectory_error(box_list, center_distance, image_size=832)
            iou_2        = mean_iou(box_list, image_size=832)

        loss_val += seq_loss.item() / valid_len / obs_length
        loss_coord_val += seq_loss_coord / valid_len / obs_length
        loss_conf_val += seq_loss_conf / valid_len / obs_length
        loss_ap += seq_ap / obs_length / valid_len

        loss_pred_val  += seq_loss_pred / valid_len / pred_length
        dis_error_val  += dis_error     / valid_len
        traj_error_val += traj_error    / valid_len
        iou_val        += iou           / valid_len
        dis_error_val_2  += dis_error_2     / valid_len
        traj_error_val_2 += traj_error_2    / valid_len
        iou_val_2        += iou_2           / valid_len
        #draw_pred_sequence_2(box_list, img2[0], pred_length, obs_length, name=f'log/flow_real_3_decoder_4/imgs/val_img_{img_num}.png')

    print(f"loss_seq: {loss_val}, loss_coord: {loss_coord_val}, loss_conf: {loss_conf_val}, mAP: {loss_ap}")

    print(f"loss_pred: {loss_pred_val}, dis_error: {dis_error_val}, traj_error: {traj_error_val}, iou: {iou_val}")
    print(f"                            dis_error: {dis_error_val_2}, traj_error: {traj_error_val_2}, iou: {iou_val_2}")




opt = Opt()
opt.useCuda = True
opt.batch_size = 1
opt.num_workers = 4
opt.image_size = 416
opt.model = YoloFlowLSTM(opt.batch_size)
opt.decoder = ConvLSTM((opt.encoding_size, opt.encoding_size), 5 * 4, 1024, opt.batch_size)
opt.pre_trained_yolo_path = 'log/flow_1/snapshot_encoder.tar'
# load model
check_point_encoder = torch.load('log/flow_real_3_decoder_4/snapshot_encoder_fine.tar', map_location=opt.device())
check_point_decoder = torch.load('log/flow_real_3_decoder_4/snapshot_decoder.tar', map_location=opt.device())
model_state_dict_yolo = check_point_encoder['model_state_dict']
opt.model.load_state_dict(model_state_dict_yolo, strict=True)
opt.decoder.load_state_dict(check_point_decoder['model_state_dict'], strict=True)
# all parameters are on the device the hidden states are not parameters
opt.model.to(opt.device())
opt.decoder.to(opt.device())
opt.yolo_loss = YoloLoss(opt.model.anchors, filter_fkt=filter_gt)
loss_params = {"grid_shape": (opt.encoding_size, opt.encoding_size),
               "image_shape": (opt.image_size, opt.image_size),
               "path_anchors": "dataset_utils/anchors/anchors5.txt"}
opt.pred_loss = NaiveLoss(loss_params)

test_flow_real_decoder(opt)

finished video 0 at index 470/470
finished video 1 at index 830/830
finished video 2 at index 660/659
finished video 3 at index 410/410
finished video 4 at index 515/513
finished video 5 at index 710/710
finished video 6 at index 590/590
finished video 0 at index 120/120
finished video 1 at index 210/210
finished video 2 at index 170/168
finished video 3 at index 105/105
finished video 4 at index 135/131
finished video 5 at index 180/180
finished video 6 at index 150/150
Mean trajectory length: 220.06725663716813
Deviation of trajectory length: 267.3380063437236
Max trajectory length: 1050.0
Min trajectory length: 1.0
Mean displacement: 96.97458052233117
Deviation of displacements: 103.94647223357384
Max displacement: 417.3182516766331
Min displacement: 0.0
###############
# VALIDATION BEGIN
###############


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:56: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:85: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


loss_seq: 4.844373292820427, loss_coord: 1.405474702376993, loss_conf: 3.4388985636854383, mAP: 0.28538651192047954
loss_pred: 0.005724765484976797, dis_error: 19.664999413233925, traj_error: 16.0879270081879, iou: 0.308637534109884
                            dis_error: 39.32999882646785, traj_error: 32.1758540163758, iou: 0.3019612422775732


TEST flow_real_4_decoder

In [28]:
def filter_gt_batch(gt):

    # used to filter only existing bb for loss
    t = gt[gt[:, :, 0] != 0.0]

    return t[:, 1:]

def test_flow_real_decoder(opt):

    dataset = MotBBImageSequence('dataset_utils/Mot17_test_single.txt', use_only_first_video=False)
    valid_data = Subset(dataset, range(dataset.valid_begin, len(dataset)))
    valid_loader = DataLoader(valid_data, batch_size=opt.batch_size, shuffle=False, num_workers=opt.num_workers, drop_last=True)

    obs_length = 10
    pred_length = 9


    print("###############")
    print("# VALIDATION BEGIN")
    print("###############")
    opt.model.eval()
    opt.decoder.eval()
    valid_len = len(valid_loader)
    loss_val = 0
    loss_ap = 0
    loss_coord_val = 0
    loss_conf_val = 0
    loss_pred_val = 0
    dis_error_val = 0
    traj_error_val = 0
    iou_val = 0
    dis_error_val_2 = 0
    traj_error_val_2 = 0
    iou_val_2 = 0
    for img_num, (gt, b, img1, img2) in enumerate(valid_loader):
        opt.model.lstm_part.reinit_lstm(opt.batch_size)
        opt.decoder.reset_hidden(opt.batch_size)

        seq_loss = 0
        seq_loss_coord, seq_loss_conf, seq_loss_pred = (0.0, 0.0, 0.0)
        seq_ap = 0
        pred_sequence = []
        for i in range(obs_length):
            single_gt = b[:, i].to(opt.device())
            single_img2 = img2[:, i].to(opt.device())
            single_img2_normed = preprocess(img2[:, i]).to(opt.device())
            single_img1_normed = preprocess(img1[:, i]).to(opt.device())

            double_image = torch.cat((single_img1_normed, single_img2_normed), dim=1)
            with torch.no_grad():
                logits = opt.model((double_image, single_img2))

                loss, loss_coord, loss_conf = opt.yolo_loss(logits, single_gt)

            seq_ap += get_ap(logits.detach(), filter_gt_batch(single_gt), opt.image_size, opt.image_size,
                             opt.model.anchors)
            seq_loss += loss
            seq_loss_conf += loss_conf.item()
            seq_loss_coord += loss_coord.item()

        with torch.no_grad():
            prev_out = logits_to_box_params(logits.detach(), opt.model.anchors)
            # the box parameters are normalized to [0, 1]
            # at the moment [batch, anchors, ...., h* w]
            # rearrange tensor s.t [batch, h, w, anchors, ...]
            # ... == x, y, w, h, conf
            # print(f"origin mask {torch.sum(mask, dim=(0, 1))}")
            prev_out = prev_out.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size,
                                     opt.encoding_size) \
                .permute(0, 3, 4, 1, 2)
            # be carefull the position of the conf/id in the targets is 0 not 4
            # so we change it at this point to be consistent with the labels

            # mask = (prev_out[:, :, :, :, 4] > 0.25).float()
            # print(f"origin mask {torch.sum(mask, dim=(0, 3))}")
            # mask = mask.unsqueeze(-1)
            prev_out = torch.Tensor(np.roll(prev_out.cpu().numpy(), 1, axis=-1)).to(opt.device())

            opt.decoder.set_hidden(opt.model.lstm_part.hidden, opt.model.lstm_part.cell)
            for i in range(pred_length):
                # todo anchor box assignment of the target boxes does not use iou
                _, yolo_target = opt.pred_loss.to_yolo(input=gt[:, obs_length].numpy(),
                                                       target=b[:, obs_length + i].numpy(), use_iou=True)
                yolo_target = torch.Tensor(yolo_target).to(opt.device())
                # target boxes are in [0, grid_h] -> normalize to 1
                # at this point i assume that the image is a square !!!
                yolo_target[:, :, :, :, 1:] = yolo_target[:, :, :, :, 1:] / opt.encoding_size
                # if i == 0:
                # print(f"target mask {torch.sum((yolo_target[:, :, :, :, 0] > 0.0).float(), dim=(0, 3))}")
                # prev_out = prev_out * mask  # todo maybe there is a better way
                input = prev_out[:, :, :, :, 1:].contiguous() \
                    .view(opt.batch_size, opt.encoding_size, opt.encoding_size,
                          len(opt.model.anchors) * 4) \
                    .permute(0, 3, 1, 2)
                pred = opt.decoder(input)
                pred = pred.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size) \
                           .permute(0, 3, 4, 1, 2)

                seq_loss_pred += opt.pred_loss.forward(pred, prev_out, yolo_target)
                pred_sequence.append((prev_out.detach().cpu().numpy(),
                                      pred.detach().cpu().numpy(),
                                      (yolo_target).detach().cpu().numpy()))
                prev_out[:, :, :, :, 1:] += pred

            seq_loss_pred = seq_loss_pred.item()
            box_list = prediction_to_box_list(pred_sequence)
            dis_error = displacement_error(box_list, center_distance)
            traj_error = mean_squared_trajectory_error(box_list, center_distance)
            iou        = mean_iou(box_list, image_size=416)
            
            dis_error_2 = displacement_error(box_list, center_distance, image_size=832)
            traj_error_2 = mean_squared_trajectory_error(box_list, center_distance, image_size=832)
            iou_2        = mean_iou(box_list, image_size=832)

        loss_val += seq_loss.item() / valid_len / obs_length
        loss_coord_val += seq_loss_coord / valid_len / obs_length
        loss_conf_val += seq_loss_conf / valid_len / obs_length
        loss_ap += seq_ap / obs_length / valid_len

        loss_pred_val  += seq_loss_pred / valid_len / pred_length
        dis_error_val  += dis_error     / valid_len
        traj_error_val += traj_error    / valid_len
        iou_val        += iou           / valid_len
        dis_error_val_2  += dis_error_2     / valid_len
        traj_error_val_2 += traj_error_2    / valid_len
        iou_val_2        += iou_2           / valid_len
        #draw_pred_sequence_2(box_list, img2[0], pred_length, obs_length, name=f'log/flow_real_4_decoder_1/imgs/val_img_{img_num}.png')

    print(f"loss_seq: {loss_val}, loss_coord: {loss_coord_val}, loss_conf: {loss_conf_val}, mAP: {loss_ap}")

    print(f"loss_pred: {loss_pred_val}, dis_error: {dis_error_val}, traj_error: {traj_error_val}, iou: {iou_val}")
    print(f"                            dis_error: {dis_error_val_2}, traj_error: {traj_error_val_2}, iou: {iou_val_2}")





opt = Opt()
opt.useCuda = True
opt.batch_size = 1
opt.num_workers = 4
opt.model = YoloFlowLSTM(opt.batch_size)
opt.decoder = ConvLSTM((opt.encoding_size, opt.encoding_size), 5 * 4, 1024, opt.batch_size)
opt.pre_trained_yolo_path = 'log/flow_1/snapshot_encoder.tar'
# load model
check_point_encoder = torch.load('log/flow_real_4_decoder_1/snapshot_encoder_fine.tar', map_location=opt.device())
check_point_decoder = torch.load('log/flow_real_4_decoder_1/snapshot_decoder.tar', map_location=opt.device())
model_state_dict_yolo = check_point_encoder['model_state_dict']
opt.model.load_state_dict(model_state_dict_yolo, strict=True)
opt.decoder.load_state_dict(check_point_decoder['model_state_dict'], strict=True)
# all parameters are on the device the hidden states are not parameters
opt.model.to(opt.device())
opt.decoder.to(opt.device())
opt.yolo_loss = YoloLoss(opt.model.anchors, filter_fkt=filter_gt)
loss_params = {"grid_shape": (opt.encoding_size, opt.encoding_size),
               "image_shape": (opt.image_size, opt.image_size),
               "path_anchors": "dataset_utils/anchors/anchors5.txt"}
opt.pred_loss = NaiveLoss(loss_params)

test_flow_real_decoder(opt)

finished video 0 at index 470/470
finished video 1 at index 830/830
finished video 2 at index 660/659
finished video 3 at index 410/410
finished video 4 at index 515/513
finished video 5 at index 710/710
finished video 6 at index 590/590
finished video 0 at index 120/120
finished video 1 at index 210/210
finished video 2 at index 170/168
finished video 3 at index 105/105
finished video 4 at index 135/131
finished video 5 at index 180/180
finished video 6 at index 150/150
Mean trajectory length: 220.06725663716813
Deviation of trajectory length: 267.3380063437236
Max trajectory length: 1050.0
Min trajectory length: 1.0
Mean displacement: 96.97458052233117
Deviation of displacements: 103.94647223357384
Max displacement: 417.3182516766331
Min displacement: 0.0
###############
# VALIDATION BEGIN
###############


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


loss_seq: 4.8705538598440015, loss_coord: 1.4605916511427641, loss_conf: 3.409962213872582, mAP: 0.2967322695304213
loss_pred: 0.005469573572669688, dis_error: 18.838315297198537, traj_error: 14.983357024449175, iou: 0.3010051411700554
                            dis_error: 37.676630594397075, traj_error: 29.96671404889835, iou: 0.2940933369657243


TEST flow_yolo

In [0]:
def filter_gt_batch(gt):

    # used to filter only existing bb for loss
    t = gt[gt[:, :, 0] != 0.0]

    return t[:, 1:]

def test_flow_real_decoder(opt):

    dataset = MotBBImageSequence('dataset_utils/Mot17_test_single.txt', use_only_first_video=False)
    valid_data = Subset(dataset, range(dataset.valid_begin, len(dataset)))
    valid_loader = DataLoader(valid_data, batch_size=opt.batch_size, shuffle=False, num_workers=opt.num_workers, drop_last=True)

    obs_length = 10
    pred_length = 9


    print("###############")
    print("# VALIDATION BEGIN")
    print("###############")
    opt.model.eval()
    opt.decoder.eval()
    valid_len = len(valid_loader)
    loss_val = 0
    loss_ap = 0
    loss_coord_val = 0
    loss_conf_val = 0
    loss_pred_val = 0
    dis_error_val = 0
    traj_error_val = 0
    for img_num, (gt, b, img1, img2) in enumerate(valid_loader):
        opt.model.lstm_part.reinit_lstm(opt.batch_size)
        opt.decoder.reset_hidden(opt.batch_size)

        seq_loss = 0
        seq_loss_coord, seq_loss_conf, seq_loss_pred = (0.0, 0.0, 0.0)
        seq_ap = 0
        pred_sequence = []
        for i in range(obs_length):
            single_gt = b[:, i].to(opt.device())
            single_img2 = img2[:, i].to(opt.device())
            single_img2_normed = preprocess(img2[:, i]).to(opt.device())
            single_img1_normed = preprocess(img1[:, i]).to(opt.device())

            double_image = torch.cat((single_img1_normed, single_img2_normed), dim=1)
            with torch.no_grad():
                logits = opt.model((double_image, single_img2))

                loss, loss_coord, loss_conf = opt.yolo_loss(logits, single_gt)

            seq_ap += get_ap(logits.detach(), filter_gt_batch(single_gt), opt.image_size, opt.image_size,
                             opt.model.anchors)
            seq_loss += loss
            seq_loss_conf += loss_conf.item()
            seq_loss_coord += loss_coord.item()

        loss_val += seq_loss.item() / valid_len / obs_length
        loss_coord_val += seq_loss_coord / valid_len / obs_length
        loss_conf_val += seq_loss_conf / valid_len / obs_length
        loss_ap += seq_ap / obs_length / valid_len

    print(f"loss_seq: {loss_val}, loss_coord: {loss_coord_val}, loss_conf: {loss_conf_val}, mAP: {loss_ap}")




opt = Opt()
opt.useCuda = True
opt.batch_size = 1
opt.num_workers = 4
opt.model = YoloFlowLSTM(opt.batch_size)
opt.decoder = ConvLSTM((opt.encoding_size, opt.encoding_size), 5 * 4, 1024, opt.batch_size)
opt.pre_trained_yolo_path = 'log/flow_1/snapshot_encoder.tar'
# load model
check_point_encoder = torch.load('log/flow_1/snapshot_encoder.tar', map_location=opt.device())
check_point_decoder = torch.load('log/flow_1/snapshot_decoder.tar', map_location=opt.device())
model_state_dict_yolo = check_point_encoder['model_state_dict']
opt.model.load_state_dict(model_state_dict_yolo, strict=True)
opt.decoder.load_state_dict(check_point_decoder['model_state_dict'])
# all parameters are on the device the hidden states are not parameters
opt.model.to(opt.device())
opt.decoder.to(opt.device())
opt.yolo_loss = YoloLoss(opt.model.anchors, filter_fkt=filter_gt)
loss_params = {"grid_shape": (opt.encoding_size, opt.encoding_size),
               "image_shape": (opt.image_size, opt.image_size),
               "path_anchors": "dataset_utils/anchors/anchors5.txt"}
opt.pred_loss = NaiveLoss(loss_params)

test_flow_real_decoder(opt)

finished video 0 at index 470/470
finished video 1 at index 830/830
finished video 2 at index 660/659
finished video 3 at index 410/410
finished video 4 at index 515/513
finished video 5 at index 710/710
finished video 6 at index 590/590
finished video 0 at index 120/120
finished video 1 at index 210/210
finished video 2 at index 170/168
finished video 3 at index 105/105
finished video 4 at index 135/131
finished video 5 at index 180/180
finished video 6 at index 150/150
Mean trajectory length: 220.06725663716813
Deviation of trajectory length: 267.3380063437236
Max trajectory length: 1050.0
Min trajectory length: 1.0
Mean displacement: 96.97458052233117
Deviation of displacements: 103.94647223357384
Max displacement: 417.3182516766331
Min displacement: 0.0
###############
# VALIDATION BEGIN
###############
loss_seq: 4.662634588826091, loss_coord: 1.4663138192187068, loss_conf: 3.196320779942054, mAP: 0.28307077395773406


 train yolo_flow_832

In [0]:
opt = Opt()
opt.num_epoches = 100
opt.image_size = 832
opt.useCuda = True
opt.learning_rate = 1e-4
opt.decay = 1e-5
opt.batch_size = 1
opt.num_workers = 4
anchors = [(0.43, 1.715), (0.745625, 3.645), (1.24375, 5.9325), (2.5, 12.24), (6.1225, 22.41375)]
opt.model = BigYoloFlowLSTM(opt.batch_size, anchors=anchors)
opt.pre_trained_yolo_path = 'log/2_yolo_832/snapshot0025.tar'
load_Snapshot_to_yolo_LSTM(opt.model, opt)
# all parameters are on the device the hidden states are not parameters
opt.model.to(opt.device())
opt.yolo_loss = YoloLoss(opt.model.anchors, filter_fkt=filter_gt)
opt.optimizer_encoder = torch.optim.Adam(opt.model.parameters(), lr=opt.learning_rate, betas=(opt.momentum, 0.999),
                                         weight_decay=opt.decay)

In [1]:
opt = Opt()
opt.num_epoches = 100
opt.image_size = 832
opt.useCuda = True
opt.learning_rate = 1e-5
opt.decay = 1e-5
opt.batch_size = 1
opt.num_workers = 4
anchors = [(0.43, 1.715), (0.745625, 3.645), (1.24375, 5.9325), (2.5, 12.24), (6.1225, 22.41375)]
opt.model = BigYoloFlowLSTM(opt.batch_size, anchors=anchors)
check_point_encoder = torch.load('log/flow_2_2/snapshot_encoder5.tar', map_location=opt.device())
model_state_dict_yolo = check_point_encoder['model_state_dict']
opt.model.load_state_dict(model_state_dict_yolo, strict=True)
# all parameters are on the device the hidden states are not parameters
opt.model.to(opt.device())
opt.yolo_loss = YoloLoss(opt.model.anchors, filter_fkt=filter_gt)

opt.optimizer_encoder = torch.optim.Adam(opt.model.parameters(), lr=opt.learning_rate, betas=(opt.momentum, 0.999),
                                         weight_decay=opt.decay)
opt.optimizer_encoder.load_state_dict(check_point_encoder['optimizer_state_dict'])
for param_group in opt.optimizer_encoder.param_groups:
    param_group['lr'] = opt.learning_rate

NameError: ignored

In [0]:
def filter_gt_batch(gt):

    # used to filter only existing bb for loss
    t = gt[gt[:, :, 0] != 0.0]

    return t[:, 1:]

def train(opt):
    # setup train and eval set
    #torch.cuda.manual_seed(123)
    #torch.manual_seed(123)
    
    # log stuff
    if os.path.isdir(opt.log_path):
        shutil.rmtree(opt.log_path)
    os.makedirs(opt.log_path)
    writer = SummaryWriter(opt.log_path)

    dataset = MotBBImageSequence('dataset_utils/Mot17_test_single.txt', use_only_first_video=False, new_width=832, new_height=832)
    train_data = Subset(dataset, range(0, dataset.valid_begin))
    valid_data = Subset(dataset, range(dataset.valid_begin, len(dataset)))
    train_loader = DataLoader(train_data, batch_size=opt.batch_size, shuffle=True, num_workers=opt.num_workers, drop_last=True)
    valid_loader = DataLoader(valid_data, batch_size=1, shuffle=False, num_workers=opt.num_workers, drop_last=True)
    
    obs_length = 10
    pred_length = 9
    
    epoch_len = len(train_loader)

    opt.model.train()
    for epoch in range(opt.num_epoches):
        for img_num, (gt, b, img1, img2) in enumerate(train_loader):
    
            opt.model.train()
        
            opt.model.lstm_part.reinit_lstm(opt.batch_size)
            
            seq_loss = 0
            seq_loss_coord, seq_loss_conf = (0.0, 0.0)
            for i in range(obs_length):  
                single_gt = b[:, i].to(opt.device())
                single_img2 = img2[:, i].to(opt.device())
                single_img2_normed = preprocess(img2[:, i]).to(opt.device())
                single_img1_normed = preprocess(img1[:, i]).to(opt.device())

                double_image = torch.cat((single_img1_normed, single_img2_normed), dim=1)

                logits = opt.model((double_image, single_img2*255.))

                loss, loss_coord, loss_conf = opt.yolo_loss(logits, single_gt)

                seq_loss += loss
                seq_loss_conf += loss_conf.item()
                seq_loss_coord += loss_coord.item()

            loss.backward()


            opt.optimizer_encoder.step()
            opt.optimizer_encoder.zero_grad()
            
            seq_loss = seq_loss.item()
            
            writeLossToSummary(writer, 'Train', seq_loss/obs_length, 
                                                seq_loss_coord / obs_length,
                                                seq_loss_conf / obs_length, 
                                                epoch * epoch_len + img_num)

            print(f"epoch:{epoch} it: {img_num}")
            print(f"loss_seq: {seq_loss/obs_length}, loss_coord: {seq_loss_coord / obs_length}, loss_conf: {seq_loss_conf / obs_length}")

              
        ###############
        # VALIDATION
        ###############
        print("###############")
        print("# VALIDATION BEGIN")
        print("###############")
        opt.model.eval()
        valid_len = len(valid_loader)
        loss_val = 0
        loss_ap = 0
        loss_coord_val = 0
        loss_conf_val = 0
        for img_num, (gt, b, img1, img2) in enumerate(valid_loader):
            opt.model.lstm_part.reinit_lstm(opt.batch_size)
            opt.optimizer_encoder.zero_grad()
            
            seq_loss = 0
            seq_loss_coord, seq_loss_conf = (0.0, 0.0)
            seq_ap = 0
            for i in range(obs_length):
                single_gt = b[:, i].to(opt.device())
                single_img2 = img2[:, i].to(opt.device())
                single_img2_normed = preprocess(img2[:, i]).to(opt.device())
                single_img1_normed = preprocess(img1[:, i]).to(opt.device())

                double_image = torch.cat((single_img1_normed, single_img2_normed), dim=1)
                with torch.no_grad():
                    logits = opt.model((double_image, single_img2*255.))

                    loss, loss_coord, loss_conf = opt.yolo_loss(logits, single_gt)
                    
                seq_ap += get_ap(logits.detach(), filter_gt_batch(single_gt), opt.image_size, opt.image_size, opt.model.anchors)
                seq_loss += loss
                seq_loss_conf += loss_conf.item()
                seq_loss_coord += loss_coord.item()

            
            loss_val += seq_loss.item() /valid_len /obs_length
            loss_coord_val += seq_loss_coord /valid_len /obs_length
            loss_conf_val += seq_loss_conf /valid_len /obs_length
            loss_ap += seq_ap / obs_length / valid_len

        writeLossToSummary(writer, 'Val', loss_val, loss_coord_val,
                           loss_conf_val, (epoch+1) * epoch_len)

        print(f"epoch:{epoch}")
        print(f"loss_seq: {loss_val}, loss_coord: {loss_coord_val}, loss_conf: {loss_conf_val}, mAP: {loss_ap}")
        writer.add_scalar('Val/mAP', loss_ap, (epoch+1) * epoch_len)

        print("###############")
        print("# VALIDATION END")
        print("###############")
        
        torch.save({'epoch': epoch, 'model_state_dict': opt.model.state_dict(),
                    'optimizer_state_dict': opt.optimizer_encoder.state_dict()},
                   opt.log_path+f'/snapshot_encoder{epoch}.tar')


    writer.close()

In [0]:
import warnings
warnings.filterwarnings('ignore')

train(opt)

pretrain yolo flow lstm on big images

In [0]:
opt = Opt()
opt.num_epoches = 100
opt.image_size = 832
opt.encoding_size = 26
opt.useCuda = True
opt.learning_rate = 1e-5
opt.decay = 1e-5
opt.batch_size = 1
opt.num_workers = 4
anchors = [(0.43, 1.715), (0.745625, 3.645), (1.24375, 5.9325), (2.5, 12.24), (6.1225, 22.41375)]
opt.model = BigYoloFlowLSTM(opt.batch_size, anchors=anchors)
opt.decoder = ConvLSTM((opt.encoding_size, opt.encoding_size), 5 * 4, 1024, opt.batch_size)

check_point_encoder = torch.load('log/flow_2_2/snapshot_encoder5.tar', map_location=opt.device())
model_state_dict = check_point_encoder['model_state_dict']
opt.model.load_state_dict(model_state_dict, strict=True)

# all parameters are on the device the hidden states are not parameters
opt.model.to(opt.device())
opt.decoder.to(opt.device())
opt.yolo_loss = YoloLoss(opt.model.anchors, filter_fkt=filter_gt)
loss_params = {"grid_shape": (opt.encoding_size, opt.encoding_size),
               "image_shape": (opt.image_size, opt.image_size),
               "path_anchors": "dataset_utils/anchors/anchors5.txt"}
opt.pred_loss = NaiveLoss(loss_params)
opt.pred_loss.anchors *= 2.0

opt.optimizer_encoder = torch.optim.Adam(opt.model.parameters(), lr=opt.learning_rate*0.1, betas=(opt.momentum, 0.999),
                                         weight_decay=opt.decay)
opt.optimizer_decoder = torch.optim.Adam(opt.decoder.parameters(), lr=1e-4, betas=(opt.momentum, 0.999),
                                         weight_decay=opt.decay)

In [0]:
opt = Opt()
opt.num_epoches = 100
opt.image_size = 832
opt.encoding_size = 26
opt.useCuda = True
opt.learning_rate = 1e-5
opt.decay = 1e-5
opt.batch_size = 1
opt.num_workers = 4
anchors = [(0.43, 1.715), (0.745625, 3.645), (1.24375, 5.9325), (2.5, 12.24), (6.1225, 22.41375)]
opt.model = BigYoloFlowLSTM(opt.batch_size, anchors=anchors)
opt.decoder = ConvLSTM((opt.encoding_size, opt.encoding_size), 5 * 4, 1024, opt.batch_size)

check_point_encoder = torch.load('log/flow_decoder_2_1/snapshot_encoder_fine.tar', map_location=opt.device())
check_point_decoder = torch.load('log/flow_decoder_2_1/snapshot_decoder.tar', map_location=opt.device())
model_state_dict = check_point_encoder['model_state_dict']
decoder_state_dict = check_point_decoder['model_state_dict']
opt.model.load_state_dict(model_state_dict, strict=True)
opt.decoder.load_state_dict(decoder_state_dict, strict=True)

# all parameters are on the device the hidden states are not parameters
opt.model.to(opt.device())
opt.decoder.to(opt.device())
opt.yolo_loss = YoloLoss(opt.model.anchors, filter_fkt=filter_gt)
loss_params = {"grid_shape": (opt.encoding_size, opt.encoding_size),
               "image_shape": (opt.image_size, opt.image_size),
               "path_anchors": "dataset_utils/anchors/anchors5.txt"}
opt.pred_loss = NaiveLoss(loss_params)
opt.pred_loss.anchors *= 2.0

opt.optimizer_encoder = torch.optim.Adam(opt.model.parameters(), lr=opt.learning_rate*0.1, betas=(opt.momentum, 0.999),
                                         weight_decay=opt.decay)
opt.optimizer_decoder = torch.optim.Adam(opt.decoder.parameters(), lr=1e-4, betas=(opt.momentum, 0.999),
                                         weight_decay=opt.decay)

opt.optimizer_encoder.load_state_dict(check_point_encoder['optimizer_state_dict'])
opt.optimizer_decoder.load_state_dict(check_point_decoder['optimizer_state_dict'])

In [0]:
opt = Opt()
opt.num_epoches = 100
opt.image_size = 832
opt.encoding_size = 26
opt.useCuda = True
opt.learning_rate = 1e-5
opt.decay = 1e-5
opt.batch_size = 1
opt.num_workers = 4
anchors = [(0.43, 1.715), (0.745625, 3.645), (1.24375, 5.9325), (2.5, 12.24), (6.1225, 22.41375)]
opt.model = BigYoloFlowLSTM(opt.batch_size, anchors=anchors)
opt.decoder = ConvLSTM((opt.encoding_size, opt.encoding_size), 5 * 4, 1024, opt.batch_size)

check_point_encoder = torch.load('log/flow_real_2_1_decoder_2/snapshot_encoder0.tar', map_location=opt.device())
check_point_decoder = torch.load('log/flow_real_2_1_decoder_2/snapshot_decoder0.tar', map_location=opt.device())
model_state_dict = check_point_encoder['model_state_dict']
decoder_state_dict = check_point_decoder['model_state_dict']
opt.model.load_state_dict(model_state_dict, strict=True)
opt.decoder.load_state_dict(decoder_state_dict, strict=True)

# all parameters are on the device the hidden states are not parameters
opt.model.to(opt.device())
opt.decoder.to(opt.device())
opt.yolo_loss = YoloLoss(opt.model.anchors, filter_fkt=filter_gt)
loss_params = {"grid_shape": (opt.encoding_size, opt.encoding_size),
               "image_shape": (opt.image_size, opt.image_size),
               "path_anchors": "dataset_utils/anchors/anchors5.txt"}
opt.pred_loss = NaiveLoss(loss_params)
opt.pred_loss.anchors *= 2.0

opt.optimizer_encoder = torch.optim.Adam(opt.model.parameters(), lr=opt.learning_rate*0.1, betas=(opt.momentum, 0.999),
                                         weight_decay=opt.decay)
opt.optimizer_decoder = torch.optim.Adam(opt.decoder.parameters(), lr=opt.learning_rate, betas=(opt.momentum, 0.999),
                                         weight_decay=opt.decay)
opt.optimizer_encoder.load_state_dict(check_point_encoder['optimizer_state_dict'])
opt.optimizer_decoder.load_state_dict(check_point_decoder['optimizer_state_dict'])


In [0]:
import random

def filter_gt_batch(gt):

    # used to filter only existing bb for loss
    t = gt[gt[:, :, 0] != 0.0]
   
    return t[:, 1:]

def train(opt):
    # setup train and eval set
    #torch.cuda.manual_seed(123)
    #torch.manual_seed(123)
    
    # log stuff
    if os.path.isdir(opt.log_path):
        shutil.rmtree(opt.log_path)
    os.makedirs(opt.log_path)
    writer = SummaryWriter(opt.log_path)

    dataset = MotBBImageSequence('dataset_utils/Mot17_test_single.txt', use_only_first_video=False, new_width=832, new_height=832)
    train_data = Subset(dataset, range(0, dataset.valid_begin))
    valid_data = Subset(dataset, range(dataset.valid_begin, len(dataset)))
    train_loader = DataLoader(train_data, batch_size=opt.batch_size, shuffle=True, num_workers=opt.num_workers, drop_last=True)
    valid_loader = DataLoader(valid_data, batch_size=opt.batch_size, shuffle=True, num_workers=opt.num_workers, drop_last=True)
    
    obs_length = 10
    pred_length = 9
    
    epoch_len = len(train_loader)
    
    for epoch in range(opt.num_epoches):
        for img_num, (gt, b, img1, img2) in enumerate(train_loader):
            opt.model.train()
            opt.decoder.train()
            opt.model.lstm_part.reinit_lstm(opt.batch_size)
            opt.decoder.reset_hidden(opt.batch_size)
            
            seq_loss = 0
            seq_ap = 0
            seq_loss_coord, seq_loss_conf, seq_loss_pred = (0.0, 0.0, 0.0)
            pred_sequence = []
            for i in range(obs_length):  
                single_gt = b[:, i].to(opt.device())
                single_img2 = img2[:, i].to(opt.device())
                single_img2_normed = preprocess(img2[:, i]).to(opt.device())
                single_img1_normed = preprocess(img1[:, i]).to(opt.device())

                double_image = torch.cat((single_img1_normed, single_img2_normed), dim=1)

                logits = opt.model((double_image, single_img2*255.))

                loss, loss_coord, loss_conf = opt.yolo_loss(logits, single_gt)
                seq_ap += get_ap(logits, filter_gt_batch(single_gt), opt.image_size, opt.image_size, opt.model.anchors)
                seq_loss += loss
                seq_loss_conf += loss_conf.item()
                seq_loss_coord += loss_coord.item()

            prev_out = logits_to_box_params(logits.detach(), opt.model.anchors)
            # the box parameters are normalized to [0, 1]
            # at the moment [batch, anchors, ...., h* w]
            # rearrange tensor s.t [batch, h, w, anchors, ...]
            # ... == x, y, w, h, conf
            #print(f"origin mask {torch.sum(mask, dim=(0, 1))}")
            prev_out = prev_out.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                               .permute(0, 3, 4, 1, 2)
            # be carefull the position of the conf/id in the targets is 0 not 4
            # so we change it at this point to be consistent with the labels

            #mask = (prev_out[:, :, :, :, 4] > 0.1).float()
            #print(f"origin mask {torch.sum(mask, dim=(0, 3))}")
            #mask = mask.unsqueeze(-1)
            prev_out = torch.Tensor(np.roll(prev_out.cpu().numpy(), 1, axis=-1)).to(opt.device())

            opt.decoder.set_hidden(opt.model.lstm_part.hidden, opt.model.lstm_part.cell)

            for i in range(pred_length):
                # todo anchor box assignment of the target boxes does not use iou
                _, yolo_target = opt.pred_loss.to_yolo(input=gt[:, obs_length].numpy(), target=b[:, obs_length + i].numpy(), use_iou=True)
                yolo_target = torch.Tensor(yolo_target).to(opt.device())
                # target boxes are in [0, grid_h] -> normalize to 1
                # at this point i assume that the image is a square !!!
                yolo_target[:, :, :, :, 1:] = yolo_target[:, :, :, :, 1:] / opt.encoding_size
                #if i == 0:
                  #print(f"target mask {torch.sum((yolo_target[:, :, :, :, 0] > 0.0).float(), dim=(0, 3))}")
                #prev_out = prev_out * mask  # todo maybe there is a better way
                #prev_out [batch, h, w, anchors, 4] 
                input = prev_out[:, :, :, :, 1:].contiguous()\
                                                .view(opt.batch_size, opt.encoding_size, opt.encoding_size, 
                                                      len(opt.model.anchors)*4)\
                                                .permute(0, 3, 1, 2)
                pred = opt.decoder(input)
                pred = pred.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                               .permute(0, 3, 4, 1, 2)

                seq_loss_pred += opt.pred_loss.forward(pred, prev_out, yolo_target)
                pred_sequence.append((prev_out.detach().cpu().numpy(),
                                      pred.detach().cpu().numpy(),
                                      (yolo_target).detach().cpu().numpy()))

                prev_out[:, :, :, :, 1:] += pred
                #prev_out[:, :, :, :, 1:] = yolo_target[:, :, :, :, 1:]

            seq_loss += seq_loss_pred
            loss = seq_loss
            loss.backward()

            opt.optimizer_encoder.step()
            opt.optimizer_decoder.step()

            opt.optimizer_encoder.zero_grad()
            opt.optimizer_decoder.zero_grad()
            
            seq_loss = seq_loss.item() - seq_loss_pred.item()
            
            writeLossToSummary(writer, 'Train', seq_loss/obs_length, 
                                                seq_loss_coord / obs_length,
                                                seq_loss_conf / obs_length, 
                                                epoch * epoch_len + img_num)

            print(f"epoch:{epoch} it: {img_num}")
            print(f"loss_seq: {seq_loss/obs_length}, loss_coord: {seq_loss_coord / obs_length}, loss_conf: {seq_loss_conf / obs_length}, mAP: {seq_ap/obs_length}")

            seq_loss_pred = seq_loss_pred.item()
            box_list = prediction_to_box_list(pred_sequence)
            dis_error = displacement_error(box_list, center_distance, image_size=832.0)
            writer.add_scalar('Train/loss_pred', seq_loss_pred / pred_length, epoch * epoch_len + img_num)
            writer.add_scalar('Train/dis_err', dis_error, epoch * epoch_len + img_num)
            writer.add_scalar('Train/AP', seq_ap/obs_length, epoch * epoch_len + img_num)
            print(f"loss_pred: {seq_loss_pred / pred_length}, dis_error: {dis_error}")

        draw_pred_sequence(box_list, img2[0], pred_length, obs_length, name='train_img.png', image_size=832)
              
        ###############
        # VALIDATION
        ###############
        print("###############")
        print("# VALIDATION BEGIN")
        print("###############")
        opt.model.eval()
        opt.decoder.eval()
        valid_len = len(valid_loader)
        loss_val = 0
        loss_ap = 0
        loss_coord_val = 0
        loss_conf_val = 0
        loss_pred_val = 0
        dis_error_val = 0
        for img_num, (gt, b, img1, img2) in enumerate(valid_loader):
            opt.model.lstm_part.reinit_lstm(opt.batch_size)
            opt.decoder.reset_hidden(opt.batch_size)
            opt.optimizer_encoder.zero_grad()
            opt.optimizer_decoder.zero_grad()
            
            seq_loss = 0
            seq_loss_coord, seq_loss_conf, seq_loss_pred = (0.0, 0.0, 0.0)
            seq_ap = 0
            pred_sequence = []
            for i in range(obs_length):
                single_gt = b[:, i].to(opt.device())
                single_img2 = img2[:, i].to(opt.device())
                single_img2_normed = preprocess(img2[:, i]).to(opt.device())
                single_img1_normed = preprocess(img1[:, i]).to(opt.device())

                double_image = torch.cat((single_img1_normed, single_img2_normed), dim=1)
                with torch.no_grad():
                    logits = opt.model((double_image, single_img2*255.))

                    loss, loss_coord, loss_conf = opt.yolo_loss(logits, single_gt)
                    
                seq_ap += get_ap(logits.detach(), filter_gt_batch(single_gt), opt.image_size, opt.image_size, opt.model.anchors)
                seq_loss += loss
                seq_loss_conf += loss_conf.item()
                seq_loss_coord += loss_coord.item()

            with torch.no_grad():
                prev_out = logits_to_box_params(logits.detach(), opt.model.anchors)
                # the box parameters are normalized to [0, 1]
                # at the moment [batch, anchors, ...., h* w]
                # rearrange tensor s.t [batch, h, w, anchors, ...]
                # ... == x, y, w, h, conf
                #print(f"origin mask {torch.sum(mask, dim=(0, 1))}")
                prev_out = prev_out.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                                   .permute(0, 3, 4, 1, 2)
                # be carefull the position of the conf/id in the targets is 0 not 4
                # so we change it at this point to be consistent with the labels

                #mask = (prev_out[:, :, :, :, 4] > 0.25).float()
                #print(f"origin mask {torch.sum(mask, dim=(0, 3))}")
                #mask = mask.unsqueeze(-1)
                prev_out = torch.Tensor(np.roll(prev_out.cpu().numpy(), 1, axis=-1)).to(opt.device())

                opt.decoder.set_hidden(opt.model.lstm_part.hidden, opt.model.lstm_part.cell)
                for i in range(pred_length):
                    # todo anchor box assignment of the target boxes does not use iou
                    _, yolo_target = opt.pred_loss.to_yolo(input=gt[:, obs_length].numpy(), target=b[:, obs_length + i].numpy(), use_iou=True)
                    yolo_target = torch.Tensor(yolo_target).to(opt.device())
                    # target boxes are in [0, grid_h] -> normalize to 1
                    # at this point i assume that the image is a square !!!
                    yolo_target[:, :, :, :, 1:] = yolo_target[:, :, :, :, 1:] / opt.encoding_size
                    #if i == 0:
                      #print(f"target mask {torch.sum((yolo_target[:, :, :, :, 0] > 0.0).float(), dim=(0, 3))}")
                    #prev_out = prev_out * mask  # todo maybe there is a better way
                    input = prev_out[:, :, :, :, 1:].contiguous()\
                                                    .view(opt.batch_size, opt.encoding_size, opt.encoding_size, 
                                                          len(opt.model.anchors)*4)\
                                                    .permute(0, 3, 1, 2)
                    pred = opt.decoder(input)
                    pred = pred.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                                   .permute(0, 3, 4, 1, 2)

                    seq_loss_pred += opt.pred_loss.forward(pred, prev_out, yolo_target)
                    pred_sequence.append((prev_out.detach().cpu().numpy(),
                                          pred.detach().cpu().numpy(),
                                          (yolo_target).detach().cpu().numpy()))
                    prev_out[:, :, :, :, 1:] += pred


                seq_loss_pred = seq_loss_pred.item()
                box_list = prediction_to_box_list(pred_sequence)
                dis_error = displacement_error(box_list, center_distance, image_size=832.0)
            
            loss_val += seq_loss.item() /valid_len /obs_length
            loss_coord_val += seq_loss_coord /valid_len /obs_length
            loss_conf_val += seq_loss_conf /valid_len /obs_length
            loss_ap += seq_ap / obs_length / valid_len

            loss_pred_val += seq_loss_pred /valid_len  / pred_length
            dis_error_val += dis_error /valid_len
        draw_pred_sequence(box_list, img2[0], pred_length, obs_length, name=f'val_img.png', image_size=832)
        writeLossToSummary(writer, 'Val', loss_val, loss_coord_val,
                           loss_conf_val, (epoch+1) * epoch_len)

        print(f"epoch:{epoch}")
        print(f"loss_seq: {loss_val}, loss_coord: {loss_coord_val}, loss_conf: {loss_conf_val}, mAP: {loss_ap}")

        box_list = prediction_to_box_list(pred_sequence)
        dis_error = displacement_error(box_list, center_distance)
        writer.add_scalar('Val/loss_pred', loss_pred_val, (epoch+1) * epoch_len)
        writer.add_scalar('Val/dis_err', dis_error_val, (epoch+1) * epoch_len)
        writer.add_scalar('Val/mAP', loss_ap, (epoch+1) * epoch_len)

        print(f"loss_pred: {loss_pred_val}, dis_error: {dis_error_val}")
        print("###############")
        print("# VALIDATION END")
        print("###############")
        
        torch.save({'epoch': epoch, 'model_state_dict': opt.model.state_dict(),
                    'optimizer_state_dict': opt.optimizer_encoder.state_dict()},
                   opt.log_path+f'/snapshot_encoder{epoch}.tar')
        torch.save({'epoch': epoch, 'model_state_dict': opt.decoder.state_dict(),
                    'optimizer_state_dict': opt.optimizer_decoder.state_dict()},
                   opt.log_path+f'/snapshot_decoder{epoch}.tar')

    writer.close()

In [7]:
import warnings
warnings.filterwarnings('ignore')

train(opt)

finished video 0 at index 470/470
finished video 1 at index 830/830
finished video 2 at index 660/659
finished video 3 at index 410/410
finished video 4 at index 515/513
finished video 5 at index 710/710
finished video 6 at index 590/590
finished video 0 at index 120/120
finished video 1 at index 210/210
finished video 2 at index 170/168
finished video 3 at index 105/105
finished video 4 at index 135/131
finished video 5 at index 180/180
finished video 6 at index 150/150
Mean trajectory length: 220.06725663716813
Deviation of trajectory length: 267.3380063437236
Max trajectory length: 1050.0
Min trajectory length: 1.0
Mean displacement: 193.94916104466233
Deviation of displacements: 207.8929444671477
Max displacement: 834.6365033532662
Min displacement: 0.0
epoch:0 it: 0
loss_seq: 1.4019540809094906, loss_coord: 0.5770532041788101, loss_conf: 0.8249009795486927, mAP: 0.9058029482407794
loss_pred: 0.002111220525370704, dis_error: 25.56536865234375
epoch:0 it: 1
loss_seq: 0.5766577683156

Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
  File "/usr/lib/p

KeyboardInterrupt: ignored

test final model

In [24]:
from dataset_utils.datasets.MotBBImageSequence import *
from torch.utils.data import DataLoader, Subset
from lstm.LSTMLosses import center_distance, displacement_error, prediction_to_box_list, draw_pred_sequence, NaiveLoss, mean_iou 
from lstm.LSTMLosses import mean_squared_trajectory_error, mean_iou
from Flow import preprocess
from yolo.yolo_LSTM import BigYoloFlowLSTM, ConvLSTM
from yolo.loss import YoloLoss
from train_yolo import Opt
from tensorboardX import SummaryWriter
import shutil
from yolo.yolo_utils import filter_non_zero_gt, get_ap, logits_to_box_params
from train_yolo import writeLossToSummary

def filter_gt_batch(gt):

    # used to filter only existing bb for loss
    t = gt[gt[:, :, 0] != 0.0]
   
    return t[:, 1:]


def test_final(opt):

    dataset = MotBBImageSequence('dataset_utils/Mot17_test_single.txt', use_only_first_video=False, new_width=832, new_height=832)
    train_data = Subset(dataset, range(0, dataset.valid_begin))
    valid_data = Subset(dataset, range(dataset.valid_begin, len(dataset)))
    valid_loader = DataLoader(valid_data, batch_size=opt.batch_size, shuffle=True, num_workers=opt.num_workers, drop_last=True)
    
    obs_length = 10
    pred_length = 9
                 
    ###############
    # VALIDATION
    ###############
    print("###############")
    print("# VALIDATION BEGIN")
    print("###############")
    opt.model.eval()
    opt.decoder.eval()
    valid_len = len(valid_loader)
    loss_val = 0
    loss_ap = 0
    loss_coord_val = 0
    loss_conf_val = 0
    loss_pred_val = 0
    dis_error_val = 0
    traj_error_val = 0
    iou_val = 0    
    for img_num, (gt, b, img1, img2) in enumerate(valid_loader):
        opt.model.lstm_part.reinit_lstm(opt.batch_size)
        opt.decoder.reset_hidden(opt.batch_size)

        seq_loss = 0
        seq_loss_coord, seq_loss_conf, seq_loss_pred = (0.0, 0.0, 0.0)
        seq_ap = 0
        pred_sequence = []
        for i in range(obs_length):
            single_gt = b[:, i].to(opt.device())
            single_img2 = img2[:, i].to(opt.device())
            single_img2_normed = preprocess(img2[:, i]).to(opt.device())
            single_img1_normed = preprocess(img1[:, i]).to(opt.device())

            double_image = torch.cat((single_img1_normed, single_img2_normed), dim=1)
            with torch.no_grad():
                logits = opt.model((double_image, single_img2*255.))

                loss, loss_coord, loss_conf = opt.yolo_loss(logits, single_gt)

            seq_ap += get_ap(logits.detach(), filter_gt_batch(single_gt), opt.image_size, opt.image_size, opt.model.anchors)
            seq_loss += loss
            seq_loss_conf += loss_conf.item()
            seq_loss_coord += loss_coord.item()

        with torch.no_grad():
            prev_out = logits_to_box_params(logits.detach(), opt.model.anchors)
            # the box parameters are normalized to [0, 1]
            # at the moment [batch, anchors, ...., h* w]
            # rearrange tensor s.t [batch, h, w, anchors, ...]
            # ... == x, y, w, h, conf
            #print(f"origin mask {torch.sum(mask, dim=(0, 1))}")
            prev_out = prev_out.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                               .permute(0, 3, 4, 1, 2)
            # be carefull the position of the conf/id in the targets is 0 not 4
            # so we change it at this point to be consistent with the labels

            #mask = (prev_out[:, :, :, :, 4] > 0.25).float()
            #print(f"origin mask {torch.sum(mask, dim=(0, 3))}")
            #mask = mask.unsqueeze(-1)
            prev_out = torch.Tensor(np.roll(prev_out.cpu().numpy(), 1, axis=-1)).to(opt.device())

            opt.decoder.set_hidden(opt.model.lstm_part.hidden, opt.model.lstm_part.cell)
            for i in range(pred_length):
                # todo anchor box assignment of the target boxes does not use iou
                _, yolo_target = opt.pred_loss.to_yolo(input=gt[:, obs_length].numpy(), target=b[:, obs_length + i].numpy(), use_iou=True)
                yolo_target = torch.Tensor(yolo_target).to(opt.device())
                # target boxes are in [0, grid_h] -> normalize to 1
                # at this point i assume that the image is a square !!!
                yolo_target[:, :, :, :, 1:] = yolo_target[:, :, :, :, 1:] / opt.encoding_size
                #if i == 0:
                  #print(f"target mask {torch.sum((yolo_target[:, :, :, :, 0] > 0.0).float(), dim=(0, 3))}")
                #prev_out = prev_out * mask  # todo maybe there is a better way
                input = prev_out[:, :, :, :, 1:].contiguous()\
                                                .view(opt.batch_size, opt.encoding_size, opt.encoding_size, 
                                                      len(opt.model.anchors)*4)\
                                                .permute(0, 3, 1, 2)
                pred = opt.decoder(input)
                pred = pred.view(opt.batch_size, len(opt.model.anchors), -1, opt.encoding_size, opt.encoding_size)\
                               .permute(0, 3, 4, 1, 2)

                seq_loss_pred += opt.pred_loss.forward(pred, prev_out, yolo_target)
                pred_sequence.append((prev_out.detach().cpu().numpy(),
                                      pred.detach().cpu().numpy(),
                                      (yolo_target).detach().cpu().numpy()))
                prev_out[:, :, :, :, 1:] += pred


            seq_loss_pred = seq_loss_pred.item()
            box_list = prediction_to_box_list(pred_sequence)
            dis_error = displacement_error(box_list, center_distance, image_size=832.0)
            traj_error = mean_squared_trajectory_error(box_list, center_distance, image_size=832.0)
            iou        = mean_iou(box_list, image_size=832.0)

        loss_val += seq_loss.item() /valid_len /obs_length
        loss_coord_val += seq_loss_coord /valid_len /obs_length
        loss_conf_val += seq_loss_conf /valid_len /obs_length
        loss_ap += seq_ap / obs_length / valid_len

        loss_pred_val += seq_loss_pred /valid_len  / pred_length
        dis_error_val += dis_error /valid_len
        traj_error_val += traj_error / valid_len
        iou_val        += iou        / valid_len
        #draw_pred_sequence(box_list, img2[0], pred_length, obs_length, name=f'log/flow_real_2_1_decoder_3/imgs/val_img{img_num}.png', image_size=832)

    print(f"loss_seq: {loss_val}, loss_coord: {loss_coord_val}, loss_conf: {loss_conf_val}, mAP: {loss_ap}")
    print(f"loss_pred: {loss_pred_val}, dis_error: {dis_error_val}, traj_error: {traj_error_val}, iou_val: {iou_val}")
    print("###############")
    print("# VALIDATION END")
    print("###############")

    
opt = Opt()
opt.num_epoches = 100
opt.image_size = 832
opt.encoding_size = 26
opt.useCuda = True
opt.batch_size = 1
opt.num_workers = 4
anchors = [(0.43, 1.715), (0.745625, 3.645), (1.24375, 5.9325), (2.5, 12.24), (6.1225, 22.41375)]
opt.model = BigYoloFlowLSTM(opt.batch_size, anchors=anchors)
opt.decoder = ConvLSTM((opt.encoding_size, opt.encoding_size), 5 * 4, 1024, opt.batch_size)

check_point_encoder = torch.load('log/flow_real_2_1_decoder_3/snapshot_encoder2.tar', map_location=opt.device())
check_point_decoder = torch.load('log/flow_real_2_1_decoder_3/snapshot_decoder2.tar', map_location=opt.device())
model_state_dict = check_point_encoder['model_state_dict']
decoder_state_dict = check_point_decoder['model_state_dict']
opt.model.load_state_dict(model_state_dict, strict=True)
opt.decoder.load_state_dict(decoder_state_dict, strict=True)

# all parameters are on the device the hidden states are not parameters
opt.model.to(opt.device())
opt.decoder.to(opt.device())
opt.yolo_loss = YoloLoss(opt.model.anchors, filter_fkt=filter_non_zero_gt)
loss_params = {"grid_shape": (opt.encoding_size, opt.encoding_size),
               "image_shape": (opt.image_size, opt.image_size),
               "path_anchors": "dataset_utils/anchors/anchors5.txt"}
opt.pred_loss = NaiveLoss(loss_params)
opt.pred_loss.anchors *= 2.0

test_final(opt)

finished video 0 at index 470/470
finished video 1 at index 830/830
finished video 2 at index 660/659
finished video 3 at index 410/410
finished video 4 at index 515/513
finished video 5 at index 710/710
finished video 6 at index 590/590
finished video 0 at index 120/120
finished video 1 at index 210/210
finished video 2 at index 170/168
finished video 3 at index 105/105
finished video 4 at index 135/131
finished video 5 at index 180/180
finished video 6 at index 150/150
Mean trajectory length: 220.06725663716813
Deviation of trajectory length: 267.3380063437236
Max trajectory length: 1050.0
Min trajectory length: 1.0
Mean displacement: 193.94916104466233
Deviation of displacements: 207.8929444671477
Max displacement: 834.6365033532662
Min displacement: 0.0
###############
# VALIDATION BEGIN
###############


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


loss_seq: 8.150001098776372, loss_coord: 1.745659509869993, loss_conf: 6.404341581632051, mAP: 0.5803965367965771
loss_pred: 0.0034639292052783426, dis_error: 25.573356454090405, traj_error: 17.799954078530753, iou_val: 0.46887123678131004
###############
# VALIDATION END
###############


finished video 0 at index 470/470
finished video 1 at index 830/830
finished video 2 at index 660/659
finished video 3 at index 410/410
finished video 4 at index 515/513
finished video 5 at index 710/710
finished video 6 at index 590/590
finished video 0 at index 120/120
finished video 1 at index 210/210
finished video 2 at index 170/168
finished video 3 at index 105/105
finished video 4 at index 135/131
finished video 5 at index 180/180
finished video 6 at index 150/150
Mean trajectory length: 220.06725663716813
Deviation of trajectory length: 267.3380063437236
Max trajectory length: 1050.0
Min trajectory length: 1.0
Mean displacement: 193.94916104466233
Deviation of displacements: 207.8929444671477
Max displacement: 834.6365033532662
Min displacement: 0.0


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


epoch:0 it: 0
loss_seq: 0.5618705529719591, loss_coord: 0.372551029920578, loss_conf: 0.18931953981518745, mAP: 0.09621212121212122
loss_pred: 0.0017682425677776337, dis_error: 49.93656921386719
epoch:0 it: 1
loss_seq: 2.1074632167816163, loss_coord: 0.6362529218196868, loss_conf: 1.471210205554962, mAP: 0.8337709870220517
loss_pred: 0.00617625978257921, dis_error: 61.37968063354492


KeyboardInterrupt: ignored